Loan word classification experiments

GPU compatible

In [1]:
import sklearn as sk
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import pandas as pd 
import numpy as np 
import os
import io
import requests
import csv
import json
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import pprint
import matplotlib.pyplot as plt
import random

In [2]:
import torch
from torch import nn
from torch import optim
from sklearn.model_selection import train_test_split

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
# device = 'cpu'
# print(device)

Using cuda device


In [19]:
pairs = None

with open('../language-pairs.json', 'r') as f:
    pairs = json.loads(f.read())
    print(pairs)
    
train_alldata = {}
test_alldata = {}

train_realdist = {}
test_realdist = {}

train_balanced = {}
test_balanced = {}

for pair in pairs:
    print(pair)
    L1 = pairs[pair]['target']['name']
    L2 = pairs[pair]['source']['name']

    # load datasets
    prefix = f'../Datasets/production_train_test/{L1}-{L2}'

    train_alldata[f'{L1}-{L2}'] = pd.read_csv(f'{prefix}/alldata/{L1}-{L2}-train_production_alldata.csv')
    test_alldata[f'{L1}-{L2}'] = pd.read_csv(f'{prefix}/alldata/{L1}-{L2}-test_production_alldata.csv')

    train_realdist[f'{L1}-{L2}'] = pd.read_csv(f'{prefix}/realdist/{L1}-{L2}-train_production_realdist.csv')
    test_realdist[f'{L1}-{L2}'] = pd.read_csv(f'{prefix}/realdist/{L1}-{L2}-test_production_realdist.csv')

    train_balanced[f'{L1}-{L2}'] = pd.read_csv(f'{prefix}/balanced/{L1}-{L2}-train_production_balanced.csv')
    test_balanced[f'{L1}-{L2}'] = pd.read_csv(f'{prefix}/balanced/{L1}-{L2}-test_production_balanced.csv')
    
    
features = ['Fast Levenshtein Distance Div Maxlen',
            'Dolgo Prime Distance Div Maxlen',
            'Feature Edit Distance Div Maxlen',
            'Hamming Feature Distance Div Maxlen',
            'Weighted Feature Distance Div Maxlen',
            'Partial Hamming Feature Distance Div Maxlen',
            'plain Levenshtein',
            'DNN_logits',
            'MBERT_cos_sim',
            'XLM_cos_sim'
           ]


# features = ['Fast Levenshtein Distance Div Maxlen',
#             'Dolgo Prime Distance Div Maxlen',
#             'Feature Edit Distance Div Maxlen',
#             'Hamming Feature Distance Div Maxlen',
#             'Weighted Feature Distance Div Maxlen',
#             'Partial Hamming Feature Distance Div Maxlen',
#             'plain Levenshtein',
             
#            ]




# features = [ 
#             'plain Levenshtein',
#             'DNN_logits',
#             'MBERT_cos_sim',
#             'XLM_cos_sim'
#            ]


# features = [ 
#             'plain Levenshtein',
             
#             'MBERT_cos_sim',
#             'XLM_cos_sim'
#            ]

# features = [ 
            
#                 'DNN_logits',
             
#             'MBERT_cos_sim',
#             'XLM_cos_sim'
#            ]
# features = [ 
#             'Partial Hamming Feature Distance Div Maxlen',
#             'plain Levenshtein',
#                 'DNN_logits',
             
#             'MBERT_cos_sim',
#             'XLM_cos_sim'
#            ]


labels = ['label_bin']

{'Hindi-Persian': {'target': {'name': 'Hindi', 'epi': 'hin-Deva', 'gtrans': 'hi', 'unicode': 'Devanagari'}, 'source': {'name': 'Persian', 'epi': 'fas-Arab', 'gtrans': 'fa', 'unicode': 'Arabic'}, 'wiki': 'https://en.m.wiktionary.org/wiki/Category:Hindi_terms_borrowed_from_Persian', 'loan-frac': '.3'}, 'English-French': {'target': {'name': 'English', 'epi': 'eng-Latn', 'gtrans': 'en', 'unicode': 'Latin'}, 'source': {'name': 'French', 'epi': 'fra-Latn', 'gtrans': 'fr', 'unicode': 'Latin'}, 'wiki': 'https://en.m.wiktionary.org/wiki/Category:English_terms_borrowed_from_French', 'loan-frac': '.3'}, 'Finnish-Swedish': {'target': {'name': 'Finnish', 'epi': 'fin-Latn', 'gtrans': 'fi', 'unicode': 'Latin'}, 'source': {'name': 'Swedish', 'epi': 'swe-Latn', 'gtrans': 'sv', 'unicode': 'Latin'}, 'wiki': 'https://en.m.wiktionary.org/wiki/Category:Finnish_terms_borrowed_from_Swedish', 'loan-frac': '.1'}, 'Kazakh-Russian': {'target': {'name': 'Kazakh', 'epi': 'kaz-Cyrl', 'gtrans': 'kk', 'unicode': 'Cyri

In [20]:
unicode_list = ["Latin", "Greek", "Cyrillic", "Arabic", "Devanagari", "Bengali", "Gurmukhi", "Tamil", "Telugu",\
                "Malayalam", "Myanmar", "Chinese"]

unicode_list = np.array(unicode_list)
if unicode_list.ndim == 1:
    unicode_list = unicode_list.reshape((-1, 1)) 
unicode_map = (unicode_list == np.unique(unicode_list)).astype(int).tolist()
unicode_map = dict(zip(unicode_list.reshape(-1,).tolist(), unicode_map))

In [21]:
for pair in pairs:
    # all data
    loan_unicode_onehots = pd.DataFrame(train_alldata[pair]['loan_unicode'].map(unicode_map).tolist(), \
                     columns = [f'loan_unicode_{l[0]}' for l in unicode_list])
    orig_unicode_onehots = pd.DataFrame(train_alldata[pair]['original_unicode'].map(unicode_map).tolist(), \
                     columns = [f'original_unicode_{l[0]}' for l in unicode_list])
    train_alldata[pair] = pd.concat([train_alldata[pair], loan_unicode_onehots, orig_unicode_onehots], axis=1)
    
    loan_unicode_onehots = pd.DataFrame(test_alldata[pair]['loan_unicode'].map(unicode_map).tolist(), \
                     columns = [f'loan_unicode_{l[0]}' for l in unicode_list])
    orig_unicode_onehots = pd.DataFrame(test_alldata[pair]['original_unicode'].map(unicode_map).tolist(), \
                     columns = [f'original_unicode_{l[0]}' for l in unicode_list])
    test_alldata[pair] = pd.concat([test_alldata[pair], loan_unicode_onehots, orig_unicode_onehots], axis=1)
    
    # real dist
    loan_unicode_onehots = pd.DataFrame(train_realdist[pair]['loan_unicode'].map(unicode_map).tolist(), \
                     columns = [f'loan_unicode_{l[0]}' for l in unicode_list])
    orig_unicode_onehots = pd.DataFrame(train_realdist[pair]['original_unicode'].map(unicode_map).tolist(), \
                     columns = [f'original_unicode_{l[0]}' for l in unicode_list])
    train_realdist[pair] = pd.concat([train_realdist[pair], loan_unicode_onehots, orig_unicode_onehots], axis=1)
                
    loan_unicode_onehots = pd.DataFrame(test_realdist[pair]['loan_unicode'].map(unicode_map).tolist(), \
                     columns = [f'loan_unicode_{l[0]}' for l in unicode_list])
    orig_unicode_onehots = pd.DataFrame(test_realdist[pair]['original_unicode'].map(unicode_map).tolist(), \
                     columns = [f'original_unicode_{l[0]}' for l in unicode_list])
    test_realdist[pair] = pd.concat([test_realdist[pair], loan_unicode_onehots, orig_unicode_onehots], axis=1)
    
    # balanced
    loan_unicode_onehots = pd.DataFrame(train_balanced[pair]['loan_unicode'].map(unicode_map).tolist(), \
                     columns = [f'loan_unicode_{l[0]}' for l in unicode_list])
    orig_unicode_onehots = pd.DataFrame(train_balanced[pair]['original_unicode'].map(unicode_map).tolist(), \
                     columns = [f'original_unicode_{l[0]}' for l in unicode_list])
    train_balanced[pair] = pd.concat([train_balanced[pair], loan_unicode_onehots, orig_unicode_onehots], axis=1)

    loan_unicode_onehots = pd.DataFrame(test_balanced[pair]['loan_unicode'].map(unicode_map).tolist(), \
                     columns = [f'loan_unicode_{l[0]}' for l in unicode_list])
    orig_unicode_onehots = pd.DataFrame(test_balanced[pair]['original_unicode'].map(unicode_map).tolist(), \
                     columns = [f'original_unicode_{l[0]}' for l in unicode_list])
    test_balanced[pair] = pd.concat([test_balanced[pair], loan_unicode_onehots, orig_unicode_onehots], axis=1)
        
features.extend([f'loan_unicode_{l[0]}' for l in unicode_list])
features.extend([f'original_unicode_{l[0]}' for l in unicode_list])

# Logistic Regression classifier

In [22]:
def classification_report_csv(report, language_pair, all_lang=False, train_evaluate_pair =True):
    #if language_pair ==True:
        
        #language_pair = pair

    report_data = []
    lines = report.split('\n')
    #print("lines",len(lines))
    #print(lines)
    for i, line in enumerate(lines[2:-3]):

        if i ==0 or i==1:
            #print("i", i)


           #print("actual lines",line)
            row = {}
            #row_data = line.split('      ')
            row_data = line.split(' ') 
            row_data = list(filter(None, row_data))
            #print(row_data)
            #print("rowdata",row_data[4] )
        row['all_language_evaluate'] = all_lang
        row['train_evaluate_pair'] = train_evaluate_pair
        row['language_pair'] = language_pair
        row['class'] = row_data[0]
        row['precision'] = float(row_data[1])
        row['recall'] = float(row_data[2])
        row['f1_score'] = float(row_data[3])
        row['support'] = float(row_data[4])

        if i==3:
            #print("actual lines", line)
            #print("i", i)
            row_data = line.split(' ') 
            row_data = list(filter(None, row_data))
            row['acc'] = float(row_data[1])

        report_data.append(row)
    print("final report", report_data[0:2])
    dataframe = pd.DataFrame.from_dict(report_data[0:2])
    return dataframe
    #dataframe.to_csv((f'{pair}-class_report.csv'), index=False)
    #class_report.to_csv((f'{pair}-class_report.csv'), index=False)

 

In [8]:
def lr_evaluate(full_train_set, full_test_set, pairs_list):
    all_result = []
    all_lang_fp_list= []
    all_lang_fn_list=[]
    evaluate_lang_fp_list = []
    evaluate_lang_fn_list = []
    train_evaluate_lang_fp_list = []
    train_evaluate_lang_fn_list = []
    weights_list = []
    
    pair ='All languages' #pair None for all languages evaluation
    
    
    
    
    display_fields = ['loan_word',
              'original_word',
              'loan_word_epitran',
              'original_word_epitran',
              'loan_english',
              'original_english'] + features
        
    train_set = pd.concat([full_train_set[pair][display_fields + ["label"] + labels] for pair in pairs])
    x_train = train_set[features].values
    x_means = np.mean(x_train, axis=0)
    x_stds = np.std(x_train, axis=0)
    x_stds[x_stds == 0] = 1
    y_train = train_set[labels].values.ravel()
    
    test_set = pd.concat([full_test_set[pair][display_fields + ["label"] + labels] for pair in pairs])
    x_test = test_set[features].values
    y_test = test_set[labels].values.ravel()
    
    # standardize input features
    x_train = (x_train - x_means)/x_stds
    
    print("Training on all langs\n")
    LR = LogisticRegression(random_state=1, solver='lbfgs', multi_class='ovr', max_iter=500).fit(x_train, y_train)

    # display regressor weights
    df = pd.DataFrame(LR.coef_, columns=features).style.set_caption('Weights')
    df_weights = pd.DataFrame(LR.coef_, columns=features) 
     
    df_weights.insert(0, 'language_pair', pair)
    
    weights_list.append(df_weights)
     
    display(df)
    print()
    
    print("Evaluating on all langs")
    x_test = (x_test - x_means)/x_stds
    y_pred = LR.predict(x_test)

    print("f1-score : ", f1_score(y_test, y_pred ))
    print("precision : ",precision_score(y_test, y_pred))
    print("recall : ",recall_score(y_test, y_pred )) 
    print("accuracy : ",accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred, output_dict=True))
    #class_report = classification_report(y_test, y_pred)
    #a = classification_report_csv(class_report,pair , all_lang=True)
    print(confusion_matrix(y_test, y_pred))
    class_report = classification_report(y_test, y_pred)
    a = classification_report_csv(class_report,pair,all_lang=True, train_evaluate_pair =False)
    all_result.append(a)
    
    #all_result.append(a)
    #class_report = classification_report(y_test, y_pred, output_dict=True)
     
    print()
    
    # display false positives and false negatives
    unq = np.array([x + 2*y for x, y in zip(y_pred, y_test)])
    tp = np.array(np.where(unq == 3)).tolist()[0]
    fp = np.array(np.where(unq == 1)).tolist()[0]
    tn = np.array(np.where(unq == 0)).tolist()[0]
    fn = np.array(np.where(unq == 2)).tolist()[0]

    #display(test_set.reset_index(drop=True).iloc[fp,:][display_fields + ["label"]].style.set_caption('False positives'))
    #display(test_set.reset_index(drop=True).iloc[fn,:][display_fields + ["label"]].style.set_caption('False negatives'))
    all_lang_fp = test_set.reset_index(drop=True).iloc[fp,:][display_fields + ["label"]]
    all_lang_fn = test_set.reset_index(drop=True).iloc[fn,:][display_fields + ["label"]]
    all_lang_fp.insert(0, 'language_pair', pair)
    all_lang_fn.insert(0, 'language_pair', pair)
    all_lang_fp_list.append(all_lang_fp)
    all_lang_fn_list.append(all_lang_fn)
    for pair in pairs_list:
        print(f'Evaluating on {pair}')
        
        test_set = full_test_set[pair][display_fields + ["label"] + labels]
        x_test = test_set[features].values
        y_test = test_set[labels].values.ravel()
        
        x_test = (x_test - x_means)/x_stds
        y_pred = LR.predict(x_test)
        
        print("f1-score : ", f1_score(y_test, y_pred ))
        print("precision : ",precision_score(y_test, y_pred))
        print("recall : ",recall_score(y_test, y_pred )) 
        print("accuracy : ",accuracy_score(y_test, y_pred))
        print(classification_report(y_test, y_pred))
        class_report = classification_report(y_test, y_pred)
        a = classification_report_csv(class_report,pair,all_lang=False, train_evaluate_pair =False)
        all_result.append(a)
        
        #class_report = pd.DataFrame(class_report).transpose()
        #class_report.to_csv('lr_alllang_report.csv', index=False)
        
        print(confusion_matrix(y_test, y_pred))
        print()
        
        # display false positives and false negatives
        unq = np.array([x + 2*y for x, y in zip(y_pred, y_test)])
        tp = np.array(np.where(unq == 3)).tolist()[0]
        fp = np.array(np.where(unq == 1)).tolist()[0]
        tn = np.array(np.where(unq == 0)).tolist()[0]
        fn = np.array(np.where(unq == 2)).tolist()[0]

        #display(test_set.reset_index(drop=True).iloc[fp,:][display_fields + ["label"]].style.set_caption('False positives'))
        #display(test_set.reset_index(drop=True).iloc[fn,:][display_fields + ["label"]].style.set_caption('False negatives'))
        evaluate_lang_fp = test_set.reset_index(drop=True).iloc[fp,:][display_fields + ["label"]]
        evaluate_lang_fn = test_set.reset_index(drop=True).iloc[fn,:][display_fields + ["label"]]
        evaluate_lang_fp.insert(0, 'language_pair', pair)
        evaluate_lang_fn.insert(0, 'language_pair', pair)
        evaluate_lang_fp_list.append(evaluate_lang_fp)
        evaluate_lang_fn_list.append(evaluate_lang_fn)
        
    for pair in pairs_list:
        print(f'Training and evaluating on {pair}')
        
        train_set = full_train_set[pair][display_fields + ["label"] + labels]
        x_train = train_set[features].values
        x_means = np.mean(x_train, axis=0)
        x_stds = np.std(x_train, axis=0)
        x_stds[x_stds == 0] = 1
        y_train = train_set[labels].values.ravel()
        
        test_set = full_test_set[pair][display_fields + ["label"] + labels]
        x_test = test_set[features].values
        y_test = test_set[labels].values.ravel()
        
        x_train = (x_train - x_means)/x_stds
        
        LR = LogisticRegression(random_state=1, solver='lbfgs', multi_class='ovr', max_iter=500).fit(x_train, y_train)
        
        # display regressor weights
        df = pd.DataFrame(LR.coef_, columns=features).style.set_caption('Weights')
        df_weights = pd.DataFrame(LR.coef_, columns=features)
        df_weights.insert(0, 'language_pair', pair)
        weights_list.append(df_weights)
        display(df)

        x_test = (x_test - x_means)/x_stds
        y_pred = LR.predict(x_test)
        
        print("f1-score : ", f1_score(y_test, y_pred ))
        print("precision : ",precision_score(y_test, y_pred))
        print("recall : ",recall_score(y_test, y_pred )) 
        print("accuracy : ",accuracy_score(y_test, y_pred))
        #print(classification_report(y_test, y_pred))
        print(classification_report(y_test, y_pred, output_dict=True))
        #class_report = classification_report(y_test, y_pred, output_dict=True)
        #class_report = pd.DataFrame(class_report).transpose()
        #class_report.to_csv((f'{pair}-class_report.csv'), index=False)
        print(confusion_matrix(y_test, y_pred))
        class_report = classification_report(y_test, y_pred)
        a = classification_report_csv(class_report,pair,all_lang=False,train_evaluate_pair =True)
        all_result.append(a)
        print()
        
        # display false positives and false negatives
        unq = np.array([x + 2*y for x, y in zip(y_pred, y_test)])
        tp = np.array(np.where(unq == 3)).tolist()[0]
        fp = np.array(np.where(unq == 1)).tolist()[0]
        tn = np.array(np.where(unq == 0)).tolist()[0]
        fn = np.array(np.where(unq == 2)).tolist()[0]

        #display(test_set.reset_index(drop=True).iloc[fp,:][display_fields + ["label"]].style.set_caption('False positives'))
        #display(test_set.reset_index(drop=True).iloc[fn,:][display_fields + ["label"]].style.set_caption('False negatives'))
        train_evaluate_lang_fp = test_set.reset_index(drop=True).iloc[fp,:][display_fields + ["label"]]
        train_evaluate_lang_fn = test_set.reset_index(drop=True).iloc[fn,:][display_fields + ["label"]]
        train_evaluate_lang_fp.insert(0, 'language_pair', pair)
        train_evaluate_lang_fn.insert(0, 'language_pair', pair)
        train_evaluate_lang_fp_list.append(train_evaluate_lang_fp)
        train_evaluate_lang_fn_list.append(train_evaluate_lang_fn)
    final_result = pd.concat(all_result, ignore_index=True)
    all_lang_fp_list = pd.concat(all_lang_fp_list, ignore_index=True)
    all_lang_fn_list = pd.concat(all_lang_fn_list, ignore_index=True)
    evaluate_lang_fp_list = pd.concat(evaluate_lang_fp_list, ignore_index=True)
    evaluate_lang_fn_list = pd.concat(evaluate_lang_fn_list, ignore_index=True)
    train_evaluate_lang_fp_list = pd.concat(train_evaluate_lang_fp_list, ignore_index=True)
    train_evaluate_lang_fn_list = pd.concat(train_evaluate_lang_fn_list, ignore_index=True)
    weights = pd.concat(weights_list, ignore_index=True)
    
    return final_result, all_lang_fp_list, all_lang_fn_list, evaluate_lang_fp_list, evaluate_lang_fn_list, train_evaluate_lang_fp_list, train_evaluate_lang_fn_list, weights

# Saving final results for alldata

In [ ]:
print("Evaluating and Saving final results as csv files on alldata splits\n")
output_alldata,all_lang_fp, all_lang_fn, evaluate_lang_fp, evaluate_lang_fn, train_evaluate_lang_fp, train_evaluate_lang_fn, weights = lr_evaluate(train_alldata, test_alldata, pairs)
#prefix = f'../Final_results/all_data/LR_results'
prefix = f'../Final_results/all_data/features_wo_dnnlogits_cosims/LR_results'

if os.path.isdir(prefix):
    print("Exists")
else:
    print("Doesn't exist")
    os.mkdir(prefix)
output_alldata.to_csv(f'{prefix}/final_results.csv')
all_lang_fp.to_csv(f'{prefix}/all_lang_falsepos.csv')
all_lang_fn.to_csv(f'{prefix}/all_lang_falseneg.csv')
evaluate_lang_fp.to_csv(f'{prefix}/all_lang_pair_eval_falsepos.csv')
evaluate_lang_fn.to_csv(f'{prefix}/all_lang_pair_eval_falseneg.csv')
train_evaluate_lang_fp.to_csv(f'{prefix}/indiv_pair_eval_falsepos.csv')
train_evaluate_lang_fn.to_csv(f'{prefix}/indiv_pair_eval_falseneg.csv')
weights.to_csv(f'{prefix}/LR_weights.csv')

# print("Evaluating on realdist splits\n")
# lr_evaluate(train_realdist, test_realdist, pairs)

# print("Evaluating on balanced splits\n")
# lr_evaluate(train_balanced, test_balanced, pairs)

# Saving final results for realdist

In [ ]:
print("Evaluating and Saving final results as csv files on realdist splits\n")
output_realdist,all_lang_fp, all_lang_fn, evaluate_lang_fp, evaluate_lang_fn, train_evaluate_lang_fp, train_evaluate_lang_fn, weights = lr_evaluate(train_realdist, test_realdist, pairs)
#prefix = f'../Final_results/real_dist/LR_results'
prefix = f'../Final_results/real_dist/features_wo_dnnlogits_cosims/LR_results'
if os.path.isdir(prefix):
    print("Exists")
else:
    print("Doesn't exist")
    os.mkdir(prefix)
output_realdist.to_csv(f'{prefix}/final_results.csv')
all_lang_fp.to_csv(f'{prefix}/all_lang_falsepos.csv')
all_lang_fn.to_csv(f'{prefix}/all_lang_falseneg.csv')
evaluate_lang_fp.to_csv(f'{prefix}/all_lang_pair_eval_falsepos.csv')
evaluate_lang_fn.to_csv(f'{prefix}/all_lang_pair_eval_falseneg.csv')
train_evaluate_lang_fp.to_csv(f'{prefix}/indiv_pair_eval_falsepos.csv')
train_evaluate_lang_fn.to_csv(f'{prefix}/indiv_pair_eval_falseneg.csv')
weights.to_csv(f'{prefix}/LR_weights.csv')

# Saving final results for balanced

In [ ]:
print("Evaluating and Saving final results as csv files on balanced splits\n")
output_balanced,all_lang_fp, all_lang_fn, evaluate_lang_fp, evaluate_lang_fn, train_evaluate_lang_fp, train_evaluate_lang_fn, weights = lr_evaluate(train_balanced, test_balanced, pairs)
#prefix = f'../Final_results/balanced/LR_results'
prefix = f'../Final_results/balanced/features_wo_dnnlogits_cosims/LR_results'
if os.path.isdir(prefix):
    print("Exists")
else:
    print("Doesn't exist")
    os.mkdir(prefix)
output_balanced.to_csv(f'{prefix}/final_results.csv')
all_lang_fp.to_csv(f'{prefix}/all_lang_falsepos.csv')
all_lang_fn.to_csv(f'{prefix}/all_lang_falseneg.csv')
evaluate_lang_fp.to_csv(f'{prefix}/all_lang_pair_eval_falsepos.csv')
evaluate_lang_fn.to_csv(f'{prefix}/all_lang_pair_eval_falseneg.csv')
train_evaluate_lang_fp.to_csv(f'{prefix}/indiv_pair_eval_falsepos.csv')
train_evaluate_lang_fn.to_csv(f'{prefix}/indiv_pair_eval_falseneg.csv')
weights.to_csv(f'{prefix}/LR_weights.csv')

# Neural Network classifier

## Definition

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, n_features):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(n_features, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(128, 1),
            
        )

    def forward(self, x):
        logits_new = self.linear_relu_stack(x)
        logits  = logits_new
        
        return torch.sigmoid(logits), logits_new
    
    def fit(self, X_train, Y_train, X_val, Y_val, criterion, optimizer, n_epochs=5000):
        train_losses = []
        val_losses = []
        train_accur = []
        val_accur = []

        for epoch in range(n_epochs):
            y_pred, logits = self(X_train.float())

            train_loss = criterion(y_pred, Y_train.float())

            if epoch % (n_epochs // 50) == 0:
                train_acc,_ = self.calculate_accuracy(Y_train, y_pred)

                y_val_pred = self(X_val.float())[0]

                val_loss = criterion(y_val_pred, Y_val.float())

                val_acc, total_corr = self.calculate_accuracy(Y_val, y_val_pred)

                print(f'''epoch {epoch}
                    Train set - loss: {self.round_tensor(train_loss)}, accuracy: {self.round_tensor(train_acc)} 
                    Val set - loss: {self.round_tensor(val_loss)}, accuracy: {self.round_tensor(val_acc)}''')
                
                train_losses.append(train_loss.detach().cpu().numpy())
                val_losses.append(val_loss.detach().cpu().numpy())

                val_accur.append(val_acc.detach().cpu().numpy())
                train_accur.append(train_acc.detach().cpu().numpy())

            optimizer.zero_grad()

            train_loss.backward()

            optimizer.step()
            
        return train_losses,val_losses,train_accur,val_accur
    
    def calculate_accuracy(self, y_true, y_pred):
        predicted = y_pred.ge(.5) 
        return ((y_true == predicted).sum().float() / len(y_true), (y_true == predicted).sum())
    
    def round_tensor(self, t, decimal_places=3):
        return round(t.item(), decimal_places)
    
    def plot_losses(self, train_losses, val_losses, train_accur, val_accur):
        epochs = range(1, len(train_accur) + 1)

        plt.plot(epochs, train_accur, 'bo', label='Training acc')
        plt.plot(epochs, val_accur, 'b', label='Vaidation acc')
        plt.title('Training and validation accuracy')
        plt.legend()

        plt.figure()

        plt.plot(epochs, train_losses, 'bo', label='Training loss')
        plt.plot(epochs, val_losses, 'b', label='Validation loss')
        plt.title('Training and validation loss')
        plt.legend()

        plt.show()

In [ ]:
def nn_evaluate(full_train_set, full_test_set, pairs_list):   
    all_result_nn = []
    all_result = []
    all_lang_fp_list= []
    all_lang_fn_list=[]
    evaluate_lang_fp_list = []
    evaluate_lang_fn_list = []
    train_evaluate_lang_fp_list = []
    train_evaluate_lang_fn_list = []
    
    pair =None #pair None for all languages evaluation
    display_fields = ['loan_word',
              'original_word',
              'loan_word_epitran',
              'original_word_epitran',
              'loan_english',
              'original_english'] + features
        
    train_set = pd.concat([full_train_set[pair][display_fields + ["label"] + labels] for pair in pairs])
    x_train = train_set[features].values
    x_means = np.mean(x_train, axis=0)
    x_stds = np.std(x_train, axis=0)
    x_stds[x_stds == 0] = 1
    y_train = train_set[labels].values.ravel()
    
    test_set = pd.concat([full_test_set[pair][display_fields + ["label"] + labels] for pair in pairs])
    x_test = test_set[features].values
    y_test = test_set[labels].values.ravel()
    
    # standardize input features
    x_train = (x_train - x_means)/x_stds
    
    torch.manual_seed(7)
    random.seed(7)
    np.random.seed(7)
    
    model = NeuralNetwork(x_train.shape[1]).to(device)
    print(model)
    
    criterion = nn.BCELoss().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.00001)
    #optimizer = optim.SGD(model.parameters(),lr=0.00001, momentum=0.0,  weight_decay=0.0, nesterov=False)
    
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=1, stratify=y_train)

    x_train = torch.tensor(x_train).to(device)
    x_val = torch.tensor(x_val).to(device)
    y_train = torch.tensor(y_train).reshape(-1,1).to(device)
    y_val = torch.tensor(y_val).reshape(-1,1).to(device)
        
    print("Training on all langs\n")
    train_losses, val_losses, train_accur, val_accur = \
        model.fit(x_train, y_train, x_val, y_val, criterion, optimizer, n_epochs=5000)
    model.plot_losses(train_losses,val_losses,train_accur,val_accur)
    
    model.eval()
    
    x_test = (x_test - x_means)/x_stds
    x_test = torch.tensor(x_test).to(device)

    with torch.no_grad():
        y_pred = model(torch.tensor(x_test).float())[0] > .5
        y_pred = y_pred.detach().cpu().numpy()

        print("f1-score : ", f1_score(y_test, y_pred))
        print("precision : ",precision_score(y_test, y_pred))
        print("recall : ",recall_score(y_test, y_pred)) 
        print("accuracy : ",accuracy_score(y_test, y_pred))
        print(classification_report(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))
        class_report = classification_report(y_test, y_pred)
        a = classification_report_csv(class_report,pair,all_lang=True, train_evaluate_pair =False)
        all_result_nn.append(a)
        print()
        
        # display false positives and false negatives
        unq = np.array([x + 2*y for x, y in zip(y_pred, y_test)])
        tp = np.array(np.where(unq == 3)).tolist()[0]
        fp = np.array(np.where(unq == 1)).tolist()[0]
        tn = np.array(np.where(unq == 0)).tolist()[0]
        fn = np.array(np.where(unq == 2)).tolist()[0]

        #display(test_set.reset_index(drop=True).iloc[fp,:][display_fields + ["label"]].style.set_caption('False positives'))
        #display(test_set.reset_index(drop=True).iloc[fn,:][display_fields + ["label"]].style.set_caption('False negatives'))
        all_lang_fp = test_set.reset_index(drop=True).iloc[fp,:][display_fields + ["label"]]
        all_lang_fn = test_set.reset_index(drop=True).iloc[fn,:][display_fields + ["label"]]
        
        all_lang_fp.insert(0, 'language_pair', pair)
        all_lang_fn.insert(0, 'language_pair', pair)
        all_lang_fp_list.append(all_lang_fp)
        all_lang_fn_list.append(all_lang_fn)
        for pair in pairs_list:
            print(f'Evaluating on {pair}')

            test_set = full_test_set[pair][display_fields + ["label"] + labels]
            x_test = test_set[features].values
            y_test = test_set[labels].values.ravel()

            x_test = (x_test - x_means)/x_stds
            x_test = torch.tensor(x_test).to(device)
            y_pred = model(torch.tensor(x_test).float())[0] > .5
            y_pred = y_pred.detach().cpu().numpy()
             
            print("f1-score : ", f1_score(y_test, y_pred ))
            print("precision : ",precision_score(y_test, y_pred))
            print("recall : ",recall_score(y_test, y_pred )) 
            print("accuracy : ",accuracy_score(y_test, y_pred))
            print(classification_report(y_test, y_pred))
            print(confusion_matrix(y_test, y_pred))
            class_report = classification_report(y_test, y_pred)
            a = classification_report_csv(class_report,pair,all_lang=False, train_evaluate_pair =False)
            all_result_nn.append(a)
            print()
            
            # display false positives and false negatives
            unq = np.array([x + 2*y for x, y in zip(y_pred, y_test)])
            tp = np.array(np.where(unq == 3)).tolist()[0]
            fp = np.array(np.where(unq == 1)).tolist()[0]
            tn = np.array(np.where(unq == 0)).tolist()[0]
            fn = np.array(np.where(unq == 2)).tolist()[0]

            #display(test_set.reset_index(drop=True).iloc[fp,:][display_fields + ["label"]].style.set_caption('False positives'))
            #display(test_set.reset_index(drop=True).iloc[fn,:][display_fields + ["label"]].style.set_caption('False negatives'))
            evaluate_lang_fp = test_set.reset_index(drop=True).iloc[fp,:][display_fields + ["label"]]
            evaluate_lang_fn = test_set.reset_index(drop=True).iloc[fn,:][display_fields + ["label"]]
            evaluate_lang_fp.insert(0, 'language_pair', pair)
            evaluate_lang_fn.insert(0, 'language_pair', pair)
            evaluate_lang_fp_list.append(evaluate_lang_fp)
            evaluate_lang_fn_list.append(evaluate_lang_fn)
    for pair in pairs_list:
        print(f'Training and evaluating on {pair}')
        
        train_set = full_train_set[pair][display_fields + ["label"] + labels]
        x_train = train_set[features].values
        x_means = np.mean(x_train, axis=0)
        x_stds = np.std(x_train, axis=0)
        x_stds[x_stds == 0] = 1
        y_train = train_set[labels].values.ravel()
        
        test_set = full_test_set[pair][display_fields + ["label"] + labels]
        x_test = test_set[features].values
        y_test = test_set[labels].values.ravel()
        
        # standardize input features
        x_train = (x_train - x_means)/x_stds
        
        model = NeuralNetwork(x_train.shape[1]).to(device)
    
        criterion = nn.BCELoss().to(device)
        optimizer = optim.Adam(model.parameters(), lr=0.00001)

        x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=1, stratify=y_train)

        x_train = torch.tensor(x_train).to(device)
        x_val = torch.tensor(x_val).to(device)
        y_train = torch.tensor(y_train).reshape(-1,1).to(device)
        y_val = torch.tensor(y_val).reshape(-1,1).to(device)

        train_losses, val_losses, train_accur, val_accur = \
            model.fit(x_train, y_train, x_val, y_val, criterion, optimizer, n_epochs=5000)
        model.plot_losses(train_losses,val_losses,train_accur,val_accur)

        model.eval()

        x_test = (x_test - x_means)/x_stds
        x_test = torch.tensor(x_test).to(device)
        
        with torch.no_grad():
            y_pred = model(torch.tensor(x_test).float())[0] > .5
            y_pred = y_pred.detach().cpu().numpy()

            print("f1-score : ", f1_score(y_test, y_pred ))
            print("precision : ",precision_score(y_test, y_pred))
            print("recall : ",recall_score(y_test, y_pred )) 
            print("accuracy : ",accuracy_score(y_test, y_pred))
            print(classification_report(y_test, y_pred))
            print(confusion_matrix(y_test, y_pred))
            class_report = classification_report(y_test, y_pred)
            a = classification_report_csv(class_report,pair,all_lang=False,train_evaluate_pair =True)
            all_result_nn.append(a)
            print()
            
            # display false positives and false negatives
            unq = np.array([x + 2*y for x, y in zip(y_pred, y_test)])
            tp = np.array(np.where(unq == 3)).tolist()[0]
            fp = np.array(np.where(unq == 1)).tolist()[0]
            tn = np.array(np.where(unq == 0)).tolist()[0]
            fn = np.array(np.where(unq == 2)).tolist()[0]

            #display(test_set.reset_index(drop=True).iloc[fp,:][display_fields + ["label"]].style.set_caption('False positives'))
            #display(test_set.reset_index(drop=True).iloc[fn,:][display_fields + ["label"]].style.set_caption('False negatives'))
            train_evaluate_lang_fp = test_set.reset_index(drop=True).iloc[fp,:][display_fields + ["label"]]
            train_evaluate_lang_fn = test_set.reset_index(drop=True).iloc[fn,:][display_fields + ["label"]]
            train_evaluate_lang_fp.insert(0, 'language_pair', pair)
            train_evaluate_lang_fn.insert(0, 'language_pair', pair)
            train_evaluate_lang_fp_list.append(train_evaluate_lang_fp)
            train_evaluate_lang_fn_list.append(train_evaluate_lang_fn)
            
    final_result = pd.concat(all_result_nn, ignore_index=True)
    all_lang_fp_list = pd.concat(all_lang_fp_list, ignore_index=True)
    all_lang_fn_list = pd.concat(all_lang_fn_list, ignore_index=True)
    evaluate_lang_fp_list = pd.concat(evaluate_lang_fp_list, ignore_index=True)
    evaluate_lang_fn_list = pd.concat(evaluate_lang_fn_list, ignore_index=True)
    train_evaluate_lang_fp_list = pd.concat(train_evaluate_lang_fp_list, ignore_index=True)
    train_evaluate_lang_fn_list = pd.concat(train_evaluate_lang_fn_list, ignore_index=True)
    
    return final_result, all_lang_fp_list, all_lang_fn_list, evaluate_lang_fp_list, evaluate_lang_fn_list, train_evaluate_lang_fp_list, train_evaluate_lang_fn_list

# NN alldata

In [ ]:
print("Evaluating on alldata splits\n")
#nn_evaluate(train_alldata, test_alldata, pairs)


print("Evaluating and Saving final results as csv files on alldata splits\n")
output_alldata,all_lang_fp, all_lang_fn, evaluate_lang_fp, evaluate_lang_fn, train_evaluate_lang_fp, train_evaluate_lang_fn = nn_evaluate(train_alldata, test_alldata, pairs)
#prefix = f'../Final_results/all_data/NN_results'

prefix = f'../Final_results/all_data/features_wo_dnnlogits_cosims/NN_results'
if os.path.isdir(prefix):
    print("Exists")
else:
    print("Doesn't exist")
    os.mkdir(prefix)
output_alldata.to_csv(f'{prefix}/final_results.csv')
all_lang_fp.to_csv(f'{prefix}/all_lang_falsepos.csv')
all_lang_fn.to_csv(f'{prefix}/all_lang_falseneg.csv')
evaluate_lang_fp.to_csv(f'{prefix}/all_lang_pair_eval_falsepos.csv')
evaluate_lang_fn.to_csv(f'{prefix}/all_lang_pair_eval_falseneg.csv')
train_evaluate_lang_fp.to_csv(f'{prefix}/indiv_pair_eval_falsepos.csv')
train_evaluate_lang_fn.to_csv(f'{prefix}/indiv_pair_eval_falseneg.csv')

# print("Evaluating on realdist splits\n")
# nn_evaluate(train_realdist, test_realdist, pairs)

# print("Evaluating on balanced splits\n")
# nn_evaluate(train_balanced, test_balanced, pairs)

# NN real dist

In [ ]:
print("Evaluating on realdist splits\n")
 
print("Evaluating and Saving final results as csv files on realdist splits\n")
output_realdist,all_lang_fp, all_lang_fn, evaluate_lang_fp, evaluate_lang_fn, train_evaluate_lang_fp, train_evaluate_lang_fn = nn_evaluate(train_realdist, test_realdist, pairs)
#prefix = f'../Final_results/real_dist/NN_results'
prefix = f'../Final_results/real_dist/features_wo_dnnlogits_cosims/NN_results'
if os.path.isdir(prefix):
    print("Exists")
else:
    print("Doesn't exist")
    os.mkdir(prefix)
output_realdist.to_csv(f'{prefix}/final_results.csv')
all_lang_fp.to_csv(f'{prefix}/all_lang_falsepos.csv')
all_lang_fn.to_csv(f'{prefix}/all_lang_falseneg.csv')
evaluate_lang_fp.to_csv(f'{prefix}/all_lang_pair_eval_falsepos.csv')
evaluate_lang_fn.to_csv(f'{prefix}/all_lang_pair_eval_falseneg.csv')
train_evaluate_lang_fp.to_csv(f'{prefix}/indiv_pair_eval_falsepos.csv')
train_evaluate_lang_fn.to_csv(f'{prefix}/indiv_pair_eval_falseneg.csv')

# NN balanced

In [ ]:
print("Evaluating on balanced splits\n")
 
print("Evaluating and Saving final results as csv files on balanced splits\n")
output_balanced,all_lang_fp, all_lang_fn, evaluate_lang_fp, evaluate_lang_fn, train_evaluate_lang_fp, train_evaluate_lang_fn = nn_evaluate(train_balanced, test_balanced, pairs)
#prefix = f'../Final_results/balanced/NN_results'
prefix = f'../Final_results/balanced/features_wo_dnnlogits_cosims/NN_results'
if os.path.isdir(prefix):
    print("Exists")
else:
    print("Doesn't exist")
    os.mkdir(prefix)
output_balanced.to_csv(f'{prefix}/final_results.csv')
all_lang_fp.to_csv(f'{prefix}/all_lang_falsepos.csv')
all_lang_fn.to_csv(f'{prefix}/all_lang_falseneg.csv')
evaluate_lang_fp.to_csv(f'{prefix}/all_lang_pair_eval_falsepos.csv')
evaluate_lang_fn.to_csv(f'{prefix}/all_lang_pair_eval_falseneg.csv')
train_evaluate_lang_fp.to_csv(f'{prefix}/indiv_pair_eval_falsepos.csv')
train_evaluate_lang_fn.to_csv(f'{prefix}/indiv_pair_eval_falseneg.csv')

# final results Pruned list

In [ ]:
train_realdist_pruned = {}

# remove any pairs in alldata testing set from realdist training set
for lang in train_realdist:
    train_realdist_pruned[lang] = train_realdist[lang][~train_realdist[lang].set_index(['loan_word','original_word']).index.\
                          isin(test_alldata[lang].set_index(['loan_word','original_word']).index)]

print("Evaluating on realdist (train) and alldata (test)\n")
output_pruned,all_lang_fp, all_lang_fn, evaluate_lang_fp, evaluate_lang_fn, train_evaluate_lang_fp, train_evaluate_lang_fn = nn_evaluate(train_realdist_pruned, test_alldata, pairs)
#prefix = f'../Final_results/pruned_nn'

prefix = f'../Final_results/pruned_nn/features_wo_dnnlogits_cosims/NN_results'

if os.path.isdir(prefix):
    print("Exists")
else:
    print("Doesn't exist")
    os.mkdir(prefix)
output_pruned.to_csv(f'{prefix}/final_results.csv')
all_lang_fp.to_csv(f'{prefix}/all_lang_falsepos.csv')
all_lang_fn.to_csv(f'{prefix}/all_lang_falseneg.csv')
evaluate_lang_fp.to_csv(f'{prefix}/all_lang_pair_eval_falsepos.csv')
evaluate_lang_fn.to_csv(f'{prefix}/all_lang_pair_eval_falseneg.csv')
train_evaluate_lang_fp.to_csv(f'{prefix}/indiv_pair_eval_falsepos.csv')
train_evaluate_lang_fn.to_csv(f'{prefix}/indiv_pair_eval_falseneg.csv')





In [ ]:
def nn_evaluate_holdout(full_train_set, full_holdout_test_set, holdout_pairs_list):  
    all_result_nn_holdout = []
    hold_out_fp=[]
    hold_out_fn =[]
    pair =None #pair None for all languages evaluation
    display_fields = ['loan_word',
              'original_word',
              'loan_word_epitran',
              'original_word_epitran',
              'loan_english',
              'original_english'] + features
        
    train_set = pd.concat([full_train_set[pair][display_fields + ["label"] + labels] for pair in pairs])
    x_train = train_set[features].values
    x_means = np.mean(x_train, axis=0)
    x_stds = np.std(x_train, axis=0)
    x_stds[x_stds == 0] = 1
    y_train = train_set[labels].values.ravel()
    
    # standardize input features
    x_train = (x_train - x_means)/x_stds
    
    torch.manual_seed(7)
    random.seed(7)
    np.random.seed(7)
    
    model = NeuralNetwork(x_train.shape[1]).to(device)
    print(model)
    
    criterion = nn.BCELoss().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.00001)
    #optimizer = optim.SGD(model.parameters(),lr=0.00001, momentum=0.0,  weight_decay=0.0, nesterov=False)
    
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=1, stratify=y_train)

    x_train = torch.tensor(x_train).to(device)
    x_val = torch.tensor(x_val).to(device)
    y_train = torch.tensor(y_train).reshape(-1,1).to(device)
    y_val = torch.tensor(y_val).reshape(-1,1).to(device)
        
    print("Training on all langs\n")
    train_losses, val_losses, train_accur, val_accur = \
        model.fit(x_train, y_train, x_val, y_val, criterion, optimizer, n_epochs=5000)
    model.plot_losses(train_losses,val_losses,train_accur,val_accur)
    
    model.eval()

    with torch.no_grad():
        for pair in holdout_pairs_list:
            print(f'Evaluating on {pair}')

            test_set = full_holdout_test_set[pair][display_fields + ["label"] + labels]
            x_test = test_set[features].values
            y_test = test_set[labels].values.ravel()

            x_test = (x_test - x_means)/x_stds
            x_test = torch.tensor(x_test).to(device)
            y_pred = model(torch.tensor(x_test).float())[0] > .5
            y_pred = y_pred.detach().cpu().numpy()
             
            print("f1-score : ", f1_score(y_test, y_pred ))
            print("precision : ",precision_score(y_test, y_pred))
            print("recall : ",recall_score(y_test, y_pred )) 
            print("accuracy : ",accuracy_score(y_test, y_pred))
            print(classification_report(y_test, y_pred))
            print(confusion_matrix(y_test, y_pred))
            class_report = classification_report(y_test, y_pred)
            a = classification_report_csv(class_report,pair,all_lang=False, train_evaluate_pair =False)
            all_result_nn_holdout.append(a)
            print()
            
            # display false positives and false negatives
            unq = np.array([x + 2*y for x, y in zip(y_pred, y_test)])
            tp = np.array(np.where(unq == 3)).tolist()[0]
            fp = np.array(np.where(unq == 1)).tolist()[0]
            tn = np.array(np.where(unq == 0)).tolist()[0]
            fn = np.array(np.where(unq == 2)).tolist()[0]

            display(test_set.reset_index(drop=True).iloc[fp,:][display_fields + ["label"]].style.set_caption('False positives'))
            display(test_set.reset_index(drop=True).iloc[fn,:][display_fields + ["label"]].style.set_caption('False negatives'))
            holdout_evaluate_lang_fp = test_set.reset_index(drop=True).iloc[fp,:][display_fields + ["label"]]
            holdout_evaluate_lang_fn = test_set.reset_index(drop=True).iloc[fn,:][display_fields + ["label"]]
            #holdout_evaluate_lang_fp['language_pair'] = pair
            #holdout_evaluate_lang_fn['language_pair'] = pair
            holdout_evaluate_lang_fp.insert(0, 'language_pair', pair)
            holdout_evaluate_lang_fn.insert(0, 'language_pair', pair)
            hold_out_fp.append(holdout_evaluate_lang_fp)
            hold_out_fn.append(holdout_evaluate_lang_fn)
    holdout_evaluate_lang_fp = pd.concat(hold_out_fp, ignore_index=True) 
    holdout_evaluate_lang_fn = pd.concat(hold_out_fn, ignore_index=True) 
    final_result = pd.concat(all_result_nn_holdout, ignore_index=True)
    return final_result, holdout_evaluate_lang_fp,holdout_evaluate_lang_fn
            

In [ ]:
with open('../language-pairs-holdout.json', 'r') as f:
    pairs_holdout = json.loads(f.read())
    print(pairs_holdout)
    
test_holdout_alldata = {}

test_holdout_realdist = {}

test_holdout_balanced = {}

for pair in pairs_holdout:
    print(pair)
    L1 = pairs_holdout[pair]['target']['name']
    L2 = pairs_holdout[pair]['source']['name']

    # load datasets
    prefix = f'../Datasets/production_train_test/{L1}-{L2}'

    test_holdout_alldata[f'{L1}-{L2}'] = pd.read_csv(f'{prefix}/alldata/{L1}-{L2}-test_production_alldata.csv')

    test_holdout_realdist[f'{L1}-{L2}'] = pd.read_csv(f'{prefix}/realdist/{L1}-{L2}-test_production_realdist.csv')

    test_holdout_balanced[f'{L1}-{L2}'] = pd.read_csv(f'{prefix}/balanced/{L1}-{L2}-test_production_balanced.csv')
    
for pair in pairs_holdout:
    # all data
    loan_unicode_onehots = pd.DataFrame(test_holdout_alldata[pair]['loan_unicode'].map(unicode_map).tolist(), \
                     columns = [f'loan_unicode_{l[0]}' for l in unicode_list])
    orig_unicode_onehots = pd.DataFrame(test_holdout_alldata[pair]['original_unicode'].map(unicode_map).tolist(), \
                     columns = [f'original_unicode_{l[0]}' for l in unicode_list])
    test_holdout_alldata[pair] = pd.concat([test_holdout_alldata[pair], loan_unicode_onehots, orig_unicode_onehots], axis=1)
    
    # real dist            
    loan_unicode_onehots = pd.DataFrame(test_holdout_realdist[pair]['loan_unicode'].map(unicode_map).tolist(), \
                     columns = [f'loan_unicode_{l[0]}' for l in unicode_list])
    orig_unicode_onehots = pd.DataFrame(test_holdout_realdist[pair]['original_unicode'].map(unicode_map).tolist(), \
                     columns = [f'original_unicode_{l[0]}' for l in unicode_list])
    test_holdout_realdist[pair] = pd.concat([test_holdout_realdist[pair], loan_unicode_onehots, orig_unicode_onehots], axis=1)
    
    # balanced
    loan_unicode_onehots = pd.DataFrame(test_holdout_balanced[pair]['loan_unicode'].map(unicode_map).tolist(), \
                     columns = [f'loan_unicode_{l[0]}' for l in unicode_list])
    orig_unicode_onehots = pd.DataFrame(test_holdout_balanced[pair]['original_unicode'].map(unicode_map).tolist(), \
                     columns = [f'original_unicode_{l[0]}' for l in unicode_list])
    test_holdout_balanced[pair] = pd.concat([test_holdout_balanced[pair], loan_unicode_onehots, orig_unicode_onehots], axis=1)
    


In [ ]:
print("Evaluating on alldata splits\n")
output_holdout, holdout_evaluate_lang_fp,holdout_evaluate_lang_fn = nn_evaluate_holdout(train_alldata, test_holdout_alldata, pairs_holdout)

#prefix = f'../Final_results/holdout/all_data'
prefix = f'../Final_results/holdout/all_data/features_wo_dnnlogits_cosims'
if os.path.isdir(prefix):
    print("Exists")
else:
    print("Doesn't exist")
    os.mkdir(prefix)
output_holdout.to_csv(f'{prefix}/holdout_final_results.csv')
holdout_evaluate_lang_fp.to_csv(f'{prefix}/holdout_falsepos.csv')
holdout_evaluate_lang_fn.to_csv(f'{prefix}/holdout_falseneg.csv')
 
# print("Evaluating on realdist splits\n")
# nn_evaluate_holdout(train_realdist, test_holdout_realdist, pairs_holdout)

# print("Evaluating on balanced splits\n")
# nn_evaluate_holdout(train_balanced, test_holdout_balanced, pairs_holdout)

In [ ]:
print("Evaluating on real dist splits\n")
output_holdout, holdout_evaluate_lang_fp,holdout_evaluate_lang_fn = nn_evaluate_holdout(train_realdist, test_holdout_realdist, pairs_holdout)

#prefix = f'../Final_results/holdout/real_dist'
prefix = f'../Final_results/holdout/real_dist/features_wo_dnnlogits_cosims'
if os.path.isdir(prefix):
    print("Exists")
else:
    print("Doesn't exist")
    os.mkdir(prefix)
output_holdout.to_csv(f'{prefix}/holdout_final_results.csv')
holdout_evaluate_lang_fp.to_csv(f'{prefix}/holdout_falsepos.csv')
holdout_evaluate_lang_fn.to_csv(f'{prefix}/holdout_falseneg.csv')

In [ ]:
print("Evaluating on balanced splits\n")
output_holdout, holdout_evaluate_lang_fp,holdout_evaluate_lang_fn = nn_evaluate_holdout(train_balanced, test_holdout_balanced, pairs_holdout)

#prefix = f'../Final_results/holdout/balanced'
prefix = f'../Final_results/holdout/balanced/features_wo_dnnlogits_cosims'
if os.path.isdir(prefix):
    print("Exists")
else:
    print("Doesn't exist")
    os.mkdir(prefix)
output_holdout.to_csv(f'{prefix}/holdout_final_results.csv')
holdout_evaluate_lang_fp.to_csv(f'{prefix}/holdout_falsepos.csv')
holdout_evaluate_lang_fn.to_csv(f'{prefix}/holdout_falseneg.csv')

# SVM Classifier

In [23]:
def svm_evaluate(full_train_set, full_test_set, pairs_list):
    pair ='All_languages'
    
    all_result = []
    all_lang_fp_list= []
    all_lang_fn_list=[]
    evaluate_lang_fp_list = []
    evaluate_lang_fn_list = []
    train_evaluate_lang_fp_list = []
    train_evaluate_lang_fn_list = []
    weights_list = []
    display_fields = ['loan_word',
              'original_word',
              'loan_word_epitran',
              'original_word_epitran',
              'loan_english',
              'original_english'] + features
        
    train_set = pd.concat([full_train_set[pair][display_fields + ["label"] + labels] for pair in pairs])
    x_train = train_set[features].values
    x_means = np.mean(x_train, axis=0)
    x_stds = np.std(x_train, axis=0)
    x_stds[x_stds == 0] = 1
    y_train = train_set[labels].values.ravel()
    
    test_set = pd.concat([full_test_set[pair][display_fields + ["label"] + labels] for pair in pairs])
    x_test = test_set[features].values
    y_test = test_set[labels].values.ravel()
    
    # standardize input features
    x_train = (x_train - x_means)/x_stds

    print("Training on all langs\n")
     
    SVM = SVC(kernel='linear')
    SVM.fit(x_train, y_train)

    # display regressor weights
    df = pd.DataFrame(SVM.coef_, columns=features).style.set_caption('Weights')
    df_weights = pd.DataFrame(SVM.coef_, columns=features) 
     
    df_weights.insert(0, 'language_pair', pair)
    
    weights_list.append(df_weights)
    display(df)
    print()
    
    print("Evaluating on all langs")
    x_test = (x_test - x_means)/x_stds
    y_pred = SVM.predict(x_test)

    print("f1-score : ", f1_score(y_test, y_pred ))
    print("precision : ",precision_score(y_test, y_pred))
    print("recall : ",recall_score(y_test, y_pred )) 
    print("accuracy : ",accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    class_report = classification_report(y_test, y_pred)
    a = classification_report_csv(class_report,pair,all_lang=True, train_evaluate_pair =False)
    all_result.append(a)
    print()
    
    # display false positives and false negatives
    unq = np.array([x + 2*y for x, y in zip(y_pred, y_test)])
    tp = np.array(np.where(unq == 3)).tolist()[0]
    fp = np.array(np.where(unq == 1)).tolist()[0]
    tn = np.array(np.where(unq == 0)).tolist()[0]
    fn = np.array(np.where(unq == 2)).tolist()[0]

    #display(test_set.reset_index(drop=True).iloc[fp,:][display_fields + ["label"]].style.set_caption('False positives'))
    #display(test_set.reset_index(drop=True).iloc[fn,:][display_fields + ["label"]].style.set_caption('False negatives'))
    all_lang_fp = test_set.reset_index(drop=True).iloc[fp,:][display_fields + ["label"]]
    all_lang_fn = test_set.reset_index(drop=True).iloc[fn,:][display_fields + ["label"]]
    
    all_lang_fp.insert(0, 'language_pair', pair)
    all_lang_fn.insert(0, 'language_pair', pair)
    all_lang_fp_list.append(all_lang_fp)
    all_lang_fn_list.append(all_lang_fn)
    
    for pair in pairs_list:
        print(f'Evaluating on {pair}')
        
        test_set = full_test_set[pair][display_fields + ["label"] + labels]
        x_test = test_set[features].values
        y_test = test_set[labels].values.ravel()
        
        x_test = (x_test - x_means)/x_stds
        y_pred = SVM.predict(x_test)
        
        print("f1-score : ", f1_score(y_test, y_pred ))
        print("precision : ",precision_score(y_test, y_pred))
        print("recall : ",recall_score(y_test, y_pred )) 
        print("accuracy : ",accuracy_score(y_test, y_pred))
        print(classification_report(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))
        class_report = classification_report(y_test, y_pred)
        a = classification_report_csv(class_report,pair,all_lang=False, train_evaluate_pair =False)
        all_result.append(a)
        print()
        
        # display false positives and false negatives
        unq = np.array([x + 2*y for x, y in zip(y_pred, y_test)])
        tp = np.array(np.where(unq == 3)).tolist()[0]
        fp = np.array(np.where(unq == 1)).tolist()[0]
        tn = np.array(np.where(unq == 0)).tolist()[0]
        fn = np.array(np.where(unq == 2)).tolist()[0]

        #display(test_set.reset_index(drop=True).iloc[fp,:][display_fields + ["label"]].style.set_caption('False positives'))
        #display(test_set.reset_index(drop=True).iloc[fn,:][display_fields + ["label"]] .style.set_caption('False negatives'))
        evaluate_lang_fp = test_set.reset_index(drop=True).iloc[fp,:][display_fields + ["label"]]
        evaluate_lang_fn = test_set.reset_index(drop=True).iloc[fn,:][display_fields + ["label"]]
        
        evaluate_lang_fp.insert(0, 'language_pair', pair)
        evaluate_lang_fn.insert(0, 'language_pair', pair)
        evaluate_lang_fp_list.append(evaluate_lang_fp)
        evaluate_lang_fn_list.append(evaluate_lang_fn)
    for pair in pairs_list:
        print(f'Training and evaluating on {pair}')
        
        train_set = full_train_set[pair][display_fields + ["label"] + labels]
        x_train = train_set[features].values
        x_means = np.mean(x_train, axis=0)
        x_stds = np.std(x_train, axis=0)
        x_stds[x_stds == 0] = 1
        y_train = train_set[labels].values.ravel()
        
        test_set = full_test_set[pair][display_fields + ["label"] + labels]
        x_test = test_set[features].values
        y_test = test_set[labels].values.ravel()
        
        x_train = (x_train - x_means)/x_stds
        
        SVM = SVC(kernel='linear')
        SVM.fit(x_train, y_train)
        
        # display regressor weights
        df = pd.DataFrame(SVM.coef_, columns=features).style.set_caption('Weights')
        df_weights = pd.DataFrame(SVM.coef_, columns=features) 
     
        df_weights.insert(0, 'language_pair', pair)
    
        weights_list.append(df_weights)
        display(df)

        x_test = (x_test - x_means)/x_stds
        y_pred = SVM.predict(x_test)
        
        print("f1-score : ", f1_score(y_test, y_pred ))
        print("precision : ",precision_score(y_test, y_pred))
        print("recall : ",recall_score(y_test, y_pred )) 
        print("accuracy : ",accuracy_score(y_test, y_pred))
        print(classification_report(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))
        class_report = classification_report(y_test, y_pred)
        a = classification_report_csv(class_report,pair,all_lang=False,train_evaluate_pair =True)
        all_result.append(a)
        print()
        
        # display false positives and false negatives
        unq = np.array([x + 2*y for x, y in zip(y_pred, y_test)])
        tp = np.array(np.where(unq == 3)).tolist()[0]
        fp = np.array(np.where(unq == 1)).tolist()[0]
        tn = np.array(np.where(unq == 0)).tolist()[0]
        fn = np.array(np.where(unq == 2)).tolist()[0]

        #display(test_set.reset_index(drop=True).iloc[fp,:][display_fields + ["label"]].style.set_caption('False positives'))
        #display(test_set.reset_index(drop=True).iloc[fn,:][display_fields + ["label"]].style.set_caption('False negatives'))
        train_evaluate_lang_fp = test_set.reset_index(drop=True).iloc[fp,:][display_fields + ["label"]]
        train_evaluate_lang_fn = test_set.reset_index(drop=True).iloc[fn,:][display_fields + ["label"]]
        
        train_evaluate_lang_fp.insert(0, 'language_pair', pair)
        train_evaluate_lang_fn.insert(0, 'language_pair', pair)
        train_evaluate_lang_fp_list.append(train_evaluate_lang_fp)
        train_evaluate_lang_fn_list.append(train_evaluate_lang_fn)
    
    final_result = pd.concat(all_result, ignore_index=True)
    all_lang_fp_list = pd.concat(all_lang_fp_list, ignore_index=True)
    all_lang_fn_list = pd.concat(all_lang_fn_list, ignore_index=True)
    evaluate_lang_fp_list = pd.concat(evaluate_lang_fp_list, ignore_index=True)
    evaluate_lang_fn_list = pd.concat(evaluate_lang_fn_list, ignore_index=True)
    train_evaluate_lang_fp_list = pd.concat(train_evaluate_lang_fp_list, ignore_index=True)
    train_evaluate_lang_fn_list = pd.concat(train_evaluate_lang_fn_list, ignore_index=True)
    weights = pd.concat(weights_list, ignore_index=True)
    
    return final_result, all_lang_fp_list, all_lang_fn_list, evaluate_lang_fp_list, evaluate_lang_fn_list, train_evaluate_lang_fp_list, train_evaluate_lang_fn_list, weights
        
        

In [25]:
print("Evaluating on alldata splits\n")
final_result,all_lang_fp, all_lang_fn, evaluate_lang_fp, evaluate_lang_fn, train_evaluate_lang_fp, train_evaluate_lang_fn, weights = svm_evaluate(train_alldata, test_alldata, pairs)

prefix = f'../Final_results/all_data/SVM_results'
#prefix = f'../Final_results/all_data/features_wo_dnnlogits_cosims/SVM_results'


if os.path.isdir(prefix):
    print("Exists")
else:
    print("Doesn't exist")
    os.mkdir(prefix)
final_result.to_csv(f'{prefix}/final_results.csv')
all_lang_fp.to_csv(f'{prefix}/all_lang_falsepos.csv')
all_lang_fn.to_csv(f'{prefix}/all_lang_falseneg.csv')
evaluate_lang_fp.to_csv(f'{prefix}/all_lang_pair_eval_falsepos.csv')
evaluate_lang_fn.to_csv(f'{prefix}/all_lang_pair_eval_falseneg.csv')
train_evaluate_lang_fp.to_csv(f'{prefix}/indiv_pair_eval_falsepos.csv')
train_evaluate_lang_fn.to_csv(f'{prefix}/indiv_pair_eval_falseneg.csv')   
weights.to_csv(f'{prefix}/SVM_weights.csv')
# print("Evaluating on realdist splits\n")
# svm_evaluate(train_realdist, test_realdist, pairs)

# print("Evaluating on balanced splits\n")
# svm_evaluate(train_balanced, test_balanced, pairs)

Evaluating on alldata splits

Training on all langs



,Fast Levenshtein Distance Div Maxlen,Dolgo Prime Distance Div Maxlen,Feature Edit Distance Div Maxlen,Hamming Feature Distance Div Maxlen,Weighted Feature Distance Div Maxlen,Partial Hamming Feature Distance Div Maxlen,plain Levenshtein,DNN_logits,MBERT_cos_sim,XLM_cos_sim,loan_unicode_Latin,loan_unicode_Greek,loan_unicode_Cyrillic,loan_unicode_Arabic,loan_unicode_Devanagari,loan_unicode_Bengali,loan_unicode_Gurmukhi,loan_unicode_Tamil,loan_unicode_Telugu,loan_unicode_Malayalam,loan_unicode_Myanmar,loan_unicode_Chinese,original_unicode_Latin,original_unicode_Greek,original_unicode_Cyrillic,original_unicode_Arabic,original_unicode_Devanagari,original_unicode_Bengali,original_unicode_Gurmukhi,original_unicode_Tamil,original_unicode_Telugu,original_unicode_Malayalam,original_unicode_Myanmar,original_unicode_Chinese
0,-0.110236,-0.089952,1.016421,0.697109,-0.184853,-1.368514,-0.547781,3.509347,0.090450,0.199903,0.000000,0.000000,0.188910,-0.057056,-0.080115,0.000000,0.000000,-0.012989,0.000000,0.000000,0.000000,0.000000,0.162960,0.000000,0.000000,-0.057056,0.000000,0.000000,0.000000,-0.095570,0.000000,0.000000,0.000000,0.000000



Evaluating on all langs
f1-score :  0.7466965285554311
precision :  0.8018278018278018
recall :  0.6986588432523051
accuracy :  0.9425509219281759
              precision    recall  f1-score   support

           0       0.96      0.98      0.97     17301
           1       0.80      0.70      0.75      2386

    accuracy                           0.94     19687
   macro avg       0.88      0.84      0.86     19687
weighted avg       0.94      0.94      0.94     19687

[[16889   412]
 [  719  1667]]
final report [{'all_language_evaluate': True, 'train_evaluate_pair': False, 'language_pair': 'All_languages', 'class': '0', 'precision': 0.96, 'recall': 0.98, 'f1_score': 0.97, 'support': 17301.0}, {'all_language_evaluate': True, 'train_evaluate_pair': False, 'language_pair': 'All_languages', 'class': '1', 'precision': 0.8, 'recall': 0.7, 'f1_score': 0.75, 'support': 2386.0, 'acc': 0.94}]

Evaluating on Hindi-Persian
f1-score :  0.7256637168141592
precision :  0.803921568627451
recall :  0

f1-score :  0.6900269541778977
precision :  0.7710843373493976
recall :  0.624390243902439
accuracy :  0.9313432835820895
              precision    recall  f1-score   support

           0       0.95      0.97      0.96      1470
           1       0.77      0.62      0.69       205

    accuracy                           0.93      1675
   macro avg       0.86      0.80      0.83      1675
weighted avg       0.93      0.93      0.93      1675

[[1432   38]
 [  77  128]]
final report [{'all_language_evaluate': False, 'train_evaluate_pair': False, 'language_pair': 'Polish-French', 'class': '0', 'precision': 0.95, 'recall': 0.97, 'f1_score': 0.96, 'support': 1470.0}, {'all_language_evaluate': False, 'train_evaluate_pair': False, 'language_pair': 'Polish-French', 'class': '1', 'precision': 0.77, 'recall': 0.62, 'f1_score': 0.69, 'support': 205.0, 'acc': 0.93}]

Evaluating on Indonesian-Dutch
f1-score :  0.8301886792452831
precision :  0.8333333333333334
recall :  0.8270676691729323
accura

,Fast Levenshtein Distance Div Maxlen,Dolgo Prime Distance Div Maxlen,Feature Edit Distance Div Maxlen,Hamming Feature Distance Div Maxlen,Weighted Feature Distance Div Maxlen,Partial Hamming Feature Distance Div Maxlen,plain Levenshtein,DNN_logits,MBERT_cos_sim,XLM_cos_sim,loan_unicode_Latin,loan_unicode_Greek,loan_unicode_Cyrillic,loan_unicode_Arabic,loan_unicode_Devanagari,loan_unicode_Bengali,loan_unicode_Gurmukhi,loan_unicode_Tamil,loan_unicode_Telugu,loan_unicode_Malayalam,loan_unicode_Myanmar,loan_unicode_Chinese,original_unicode_Latin,original_unicode_Greek,original_unicode_Cyrillic,original_unicode_Arabic,original_unicode_Devanagari,original_unicode_Bengali,original_unicode_Gurmukhi,original_unicode_Tamil,original_unicode_Telugu,original_unicode_Malayalam,original_unicode_Myanmar,original_unicode_Chinese
0,-0.409210,-0.060178,-0.234523,0.149537,-0.146449,-0.071039,0.363607,4.249317,-0.001123,0.022150,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


...

f1-score :  0.7333333333333334
precision :  0.772972972972973
recall :  0.697560975609756
accuracy :  0.937910447761194
              precision    recall  f1-score   support

           0       0.96      0.97      0.96      1470
           1       0.77      0.70      0.73       205

    accuracy                           0.94      1675
   macro avg       0.87      0.83      0.85      1675
weighted avg       0.94      0.94      0.94      1675

[[1428   42]
 [  62  143]]
final report [{'all_language_evaluate': False, 'train_evaluate_pair': True, 'language_pair': 'Polish-French', 'class': '0', 'precision': 0.96, 'recall': 0.97, 'f1_score': 0.96, 'support': 1470.0}, {'all_language_evaluate': False, 'train_evaluate_pair': True, 'language_pair': 'Polish-French', 'class': '1', 'precision': 0.77, 'recall': 0.7, 'f1_score': 0.73, 'support': 205.0, 'acc': 0.94}]

Training and evaluating on Indonesian-Dutch


,Fast Levenshtein Distance Div Maxlen,Dolgo Prime Distance Div Maxlen,Feature Edit Distance Div Maxlen,Hamming Feature Distance Div Maxlen,Weighted Feature Distance Div Maxlen,Partial Hamming Feature Distance Div Maxlen,plain Levenshtein,DNN_logits,MBERT_cos_sim,XLM_cos_sim,loan_unicode_Latin,loan_unicode_Greek,loan_unicode_Cyrillic,loan_unicode_Arabic,loan_unicode_Devanagari,loan_unicode_Bengali,loan_unicode_Gurmukhi,loan_unicode_Tamil,loan_unicode_Telugu,loan_unicode_Malayalam,loan_unicode_Myanmar,loan_unicode_Chinese,original_unicode_Latin,original_unicode_Greek,original_unicode_Cyrillic,original_unicode_Arabic,original_unicode_Devanagari,original_unicode_Bengali,original_unicode_Gurmukhi,original_unicode_Tamil,original_unicode_Telugu,original_unicode_Malayalam,original_unicode_Myanmar,original_unicode_Chinese
0,-0.028903,0.156673,0.029122,0.698036,-0.703871,0.352172,-1.206297,4.174040,0.067455,0.218024,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


f1-score :  0.8352490421455939
precision :  0.8515625
recall :  0.8195488721804511
accuracy :  0.9547844374342797
              precision    recall  f1-score   support

           0       0.97      0.98      0.97      1636
           1       0.85      0.82      0.84       266

    accuracy                           0.95      1902
   macro avg       0.91      0.90      0.90      1902
weighted avg       0.95      0.95      0.95      1902

[[1598   38]
 [  48  218]]
final report [{'all_language_evaluate': False, 'train_evaluate_pair': True, 'language_pair': 'Indonesian-Dutch', 'class': '0', 'precision': 0.97, 'recall': 0.98, 'f1_score': 0.97, 'support': 1636.0}, {'all_language_evaluate': False, 'train_evaluate_pair': True, 'language_pair': 'Indonesian-Dutch', 'class': '1', 'precision': 0.85, 'recall': 0.82, 'f1_score': 0.84, 'support': 266.0, 'acc': 0.95}]

Training and evaluating on Romanian-French


,Fast Levenshtein Distance Div Maxlen,Dolgo Prime Distance Div Maxlen,Feature Edit Distance Div Maxlen,Hamming Feature Distance Div Maxlen,Weighted Feature Distance Div Maxlen,Partial Hamming Feature Distance Div Maxlen,plain Levenshtein,DNN_logits,MBERT_cos_sim,XLM_cos_sim,loan_unicode_Latin,loan_unicode_Greek,loan_unicode_Cyrillic,loan_unicode_Arabic,loan_unicode_Devanagari,loan_unicode_Bengali,loan_unicode_Gurmukhi,loan_unicode_Tamil,loan_unicode_Telugu,loan_unicode_Malayalam,loan_unicode_Myanmar,loan_unicode_Chinese,original_unicode_Latin,original_unicode_Greek,original_unicode_Cyrillic,original_unicode_Arabic,original_unicode_Devanagari,original_unicode_Bengali,original_unicode_Gurmukhi,original_unicode_Tamil,original_unicode_Telugu,original_unicode_Malayalam,original_unicode_Myanmar,original_unicode_Chinese
0,-0.129567,-0.185300,-0.360038,0.253768,0.116074,0.126768,-0.919823,2.848118,0.081097,0.031072,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


f1-score :  0.7584415584415584
precision :  0.7849462365591398
recall :  0.7336683417085427
accuracy :  0.9441105769230769
              precision    recall  f1-score   support

           0       0.96      0.97      0.97      1465
           1       0.78      0.73      0.76       199

    accuracy                           0.94      1664
   macro avg       0.87      0.85      0.86      1664
weighted avg       0.94      0.94      0.94      1664

[[1425   40]
 [  53  146]]
final report [{'all_language_evaluate': False, 'train_evaluate_pair': True, 'language_pair': 'Romanian-French', 'class': '0', 'precision': 0.96, 'recall': 0.97, 'f1_score': 0.97, 'support': 1465.0}, {'all_language_evaluate': False, 'train_evaluate_pair': True, 'language_pair': 'Romanian-French', 'class': '1', 'precision': 0.78, 'recall': 0.73, 'f1_score': 0.76, 'support': 199.0, 'acc': 0.94}]

Exists


In [26]:
print("Evaluating on realdist splits\n")
final_result,all_lang_fp, all_lang_fn, evaluate_lang_fp, evaluate_lang_fn, train_evaluate_lang_fp, train_evaluate_lang_fn, weights = svm_evaluate(train_realdist, test_realdist, pairs)

prefix = f'../Final_results/real_dist/SVM_results'
#prefix = f'../Final_results/real_dist/features_wo_dnnlogits_cosims/SVM_results'
if os.path.isdir(prefix):
    print("Exists")
else:
    print("Doesn't exist")
    os.mkdir(prefix)
final_result.to_csv(f'{prefix}/final_results.csv')
all_lang_fp.to_csv(f'{prefix}/all_lang_falsepos.csv')
all_lang_fn.to_csv(f'{prefix}/all_lang_falseneg.csv')
evaluate_lang_fp.to_csv(f'{prefix}/all_lang_pair_eval_falsepos.csv')
evaluate_lang_fn.to_csv(f'{prefix}/all_lang_pair_eval_falseneg.csv')
train_evaluate_lang_fp.to_csv(f'{prefix}/indiv_pair_eval_falsepos.csv')
train_evaluate_lang_fn.to_csv(f'{prefix}/indiv_pair_eval_falseneg.csv')   
weights.to_csv(f'{prefix}/SVM_weights.csv') 

Evaluating on realdist splits

Training on all langs



,Fast Levenshtein Distance Div Maxlen,Dolgo Prime Distance Div Maxlen,Feature Edit Distance Div Maxlen,Hamming Feature Distance Div Maxlen,Weighted Feature Distance Div Maxlen,Partial Hamming Feature Distance Div Maxlen,plain Levenshtein,DNN_logits,MBERT_cos_sim,XLM_cos_sim,loan_unicode_Latin,loan_unicode_Greek,loan_unicode_Cyrillic,loan_unicode_Arabic,loan_unicode_Devanagari,loan_unicode_Bengali,loan_unicode_Gurmukhi,loan_unicode_Tamil,loan_unicode_Telugu,loan_unicode_Malayalam,loan_unicode_Myanmar,loan_unicode_Chinese,original_unicode_Latin,original_unicode_Greek,original_unicode_Cyrillic,original_unicode_Arabic,original_unicode_Devanagari,original_unicode_Bengali,original_unicode_Gurmukhi,original_unicode_Tamil,original_unicode_Telugu,original_unicode_Malayalam,original_unicode_Myanmar,original_unicode_Chinese
0,-0.207335,-0.141270,0.220731,1.289530,-0.266540,-1.095382,-0.777955,3.309823,0.008973,0.025218,0.000000,0.000000,0.161097,-0.087167,-0.035100,0.000000,0.000000,-0.014551,0.000000,0.000000,0.000000,0.000000,0.199193,0.000000,0.000000,-0.087167,0.000000,0.000000,0.000000,-0.088366,0.000000,0.000000,0.000000,0.000000



Evaluating on all langs
f1-score :  0.8398366606170599
precision :  0.9154302670623146
recall :  0.7757753562447611
accuracy :  0.9558446431921946
              precision    recall  f1-score   support

           0       0.96      0.99      0.97     13603
           1       0.92      0.78      0.84      2386

    accuracy                           0.96     15989
   macro avg       0.94      0.88      0.91     15989
weighted avg       0.95      0.96      0.95     15989

[[13432   171]
 [  535  1851]]
final report [{'all_language_evaluate': True, 'train_evaluate_pair': False, 'language_pair': 'All_languages', 'class': '0', 'precision': 0.96, 'recall': 0.99, 'f1_score': 0.97, 'support': 13603.0}, {'all_language_evaluate': True, 'train_evaluate_pair': False, 'language_pair': 'All_languages', 'class': '1', 'precision': 0.92, 'recall': 0.78, 'f1_score': 0.84, 'support': 2386.0, 'acc': 0.96}]

Evaluating on Hindi-Persian
f1-score :  0.8608695652173912
precision :  0.9339622641509434
recall :

f1-score :  0.8128342245989306
precision :  0.8994082840236687
recall :  0.7414634146341463
accuracy :  0.9658869395711501
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      1847
           1       0.90      0.74      0.81       205

    accuracy                           0.97      2052
   macro avg       0.94      0.87      0.90      2052
weighted avg       0.96      0.97      0.96      2052

[[1830   17]
 [  53  152]]
final report [{'all_language_evaluate': False, 'train_evaluate_pair': False, 'language_pair': 'Polish-French', 'class': '0', 'precision': 0.97, 'recall': 0.99, 'f1_score': 0.98, 'support': 1847.0}, {'all_language_evaluate': False, 'train_evaluate_pair': False, 'language_pair': 'Polish-French', 'class': '1', 'precision': 0.9, 'recall': 0.74, 'f1_score': 0.81, 'support': 205.0, 'acc': 0.97}]

Evaluating on Indonesian-Dutch
f1-score :  0.9227722772277228
precision :  0.9748953974895398
recall :  0.8759398496240601
accura

,Fast Levenshtein Distance Div Maxlen,Dolgo Prime Distance Div Maxlen,Feature Edit Distance Div Maxlen,Hamming Feature Distance Div Maxlen,Weighted Feature Distance Div Maxlen,Partial Hamming Feature Distance Div Maxlen,plain Levenshtein,DNN_logits,MBERT_cos_sim,XLM_cos_sim,loan_unicode_Latin,loan_unicode_Greek,loan_unicode_Cyrillic,loan_unicode_Arabic,loan_unicode_Devanagari,loan_unicode_Bengali,loan_unicode_Gurmukhi,loan_unicode_Tamil,loan_unicode_Telugu,loan_unicode_Malayalam,loan_unicode_Myanmar,loan_unicode_Chinese,original_unicode_Latin,original_unicode_Greek,original_unicode_Cyrillic,original_unicode_Arabic,original_unicode_Devanagari,original_unicode_Bengali,original_unicode_Gurmukhi,original_unicode_Tamil,original_unicode_Telugu,original_unicode_Malayalam,original_unicode_Myanmar,original_unicode_Chinese
0,-0.623771,-0.040326,-0.158854,-0.411522,-0.187851,0.393302,0.312532,3.631307,0.099157,-0.060298,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


...

f1-score :  0.8201058201058201
precision :  0.8959537572254336
recall :  0.7560975609756098
accuracy :  0.9668615984405458
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      1847
           1       0.90      0.76      0.82       205

    accuracy                           0.97      2052
   macro avg       0.93      0.87      0.90      2052
weighted avg       0.97      0.97      0.97      2052

[[1829   18]
 [  50  155]]
final report [{'all_language_evaluate': False, 'train_evaluate_pair': True, 'language_pair': 'Polish-French', 'class': '0', 'precision': 0.97, 'recall': 0.99, 'f1_score': 0.98, 'support': 1847.0}, {'all_language_evaluate': False, 'train_evaluate_pair': True, 'language_pair': 'Polish-French', 'class': '1', 'precision': 0.9, 'recall': 0.76, 'f1_score': 0.82, 'support': 205.0, 'acc': 0.97}]

Training and evaluating on Indonesian-Dutch


,Fast Levenshtein Distance Div Maxlen,Dolgo Prime Distance Div Maxlen,Feature Edit Distance Div Maxlen,Hamming Feature Distance Div Maxlen,Weighted Feature Distance Div Maxlen,Partial Hamming Feature Distance Div Maxlen,plain Levenshtein,DNN_logits,MBERT_cos_sim,XLM_cos_sim,loan_unicode_Latin,loan_unicode_Greek,loan_unicode_Cyrillic,loan_unicode_Arabic,loan_unicode_Devanagari,loan_unicode_Bengali,loan_unicode_Gurmukhi,loan_unicode_Tamil,loan_unicode_Telugu,loan_unicode_Malayalam,loan_unicode_Myanmar,loan_unicode_Chinese,original_unicode_Latin,original_unicode_Greek,original_unicode_Cyrillic,original_unicode_Arabic,original_unicode_Devanagari,original_unicode_Bengali,original_unicode_Gurmukhi,original_unicode_Tamil,original_unicode_Telugu,original_unicode_Malayalam,original_unicode_Myanmar,original_unicode_Chinese
0,-0.073134,0.042959,-0.057954,1.139004,-1.203439,0.483872,-1.675780,3.331362,0.112762,0.290611,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


f1-score :  0.946360153256705
precision :  0.96484375
recall :  0.9285714285714286
accuracy :  0.9578947368421052
              precision    recall  f1-score   support

           0       0.95      0.98      0.97       399
           1       0.96      0.93      0.95       266

    accuracy                           0.96       665
   macro avg       0.96      0.95      0.96       665
weighted avg       0.96      0.96      0.96       665

[[390   9]
 [ 19 247]]
final report [{'all_language_evaluate': False, 'train_evaluate_pair': True, 'language_pair': 'Indonesian-Dutch', 'class': '0', 'precision': 0.95, 'recall': 0.98, 'f1_score': 0.97, 'support': 399.0}, {'all_language_evaluate': False, 'train_evaluate_pair': True, 'language_pair': 'Indonesian-Dutch', 'class': '1', 'precision': 0.96, 'recall': 0.93, 'f1_score': 0.95, 'support': 266.0, 'acc': 0.96}]

Training and evaluating on Romanian-French


,Fast Levenshtein Distance Div Maxlen,Dolgo Prime Distance Div Maxlen,Feature Edit Distance Div Maxlen,Hamming Feature Distance Div Maxlen,Weighted Feature Distance Div Maxlen,Partial Hamming Feature Distance Div Maxlen,plain Levenshtein,DNN_logits,MBERT_cos_sim,XLM_cos_sim,loan_unicode_Latin,loan_unicode_Greek,loan_unicode_Cyrillic,loan_unicode_Arabic,loan_unicode_Devanagari,loan_unicode_Bengali,loan_unicode_Gurmukhi,loan_unicode_Tamil,loan_unicode_Telugu,loan_unicode_Malayalam,loan_unicode_Myanmar,loan_unicode_Chinese,original_unicode_Latin,original_unicode_Greek,original_unicode_Cyrillic,original_unicode_Arabic,original_unicode_Devanagari,original_unicode_Bengali,original_unicode_Gurmukhi,original_unicode_Tamil,original_unicode_Telugu,original_unicode_Malayalam,original_unicode_Myanmar,original_unicode_Chinese
0,-0.406783,-0.128574,0.072739,-0.109208,-0.385473,0.380011,-1.075477,2.636973,0.062783,0.023087,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


f1-score :  0.9028871391076114
precision :  0.945054945054945
recall :  0.864321608040201
accuracy :  0.9442771084337349
              precision    recall  f1-score   support

           0       0.94      0.98      0.96       465
           1       0.95      0.86      0.90       199

    accuracy                           0.94       664
   macro avg       0.94      0.92      0.93       664
weighted avg       0.94      0.94      0.94       664

[[455  10]
 [ 27 172]]
final report [{'all_language_evaluate': False, 'train_evaluate_pair': True, 'language_pair': 'Romanian-French', 'class': '0', 'precision': 0.94, 'recall': 0.98, 'f1_score': 0.96, 'support': 465.0}, {'all_language_evaluate': False, 'train_evaluate_pair': True, 'language_pair': 'Romanian-French', 'class': '1', 'precision': 0.95, 'recall': 0.86, 'f1_score': 0.9, 'support': 199.0, 'acc': 0.94}]

Exists


In [27]:
print("Evaluating on balanced splits\n")
final_result,all_lang_fp, all_lang_fn, evaluate_lang_fp, evaluate_lang_fn, train_evaluate_lang_fp, train_evaluate_lang_fn, weights = svm_evaluate(train_balanced, test_balanced, pairs)

prefix = f'../Final_results/balanced/SVM_results'
#prefix = f'../Final_results/balanced/features_wo_dnnlogits_cosims/SVM_results'
if os.path.isdir(prefix):
    print("Exists")
else:
    print("Doesn't exist")
    os.mkdir(prefix)
final_result.to_csv(f'{prefix}/final_results.csv')
all_lang_fp.to_csv(f'{prefix}/all_lang_falsepos.csv')
all_lang_fn.to_csv(f'{prefix}/all_lang_falseneg.csv')
evaluate_lang_fp.to_csv(f'{prefix}/all_lang_pair_eval_falsepos.csv')
evaluate_lang_fn.to_csv(f'{prefix}/all_lang_pair_eval_falseneg.csv')
train_evaluate_lang_fp.to_csv(f'{prefix}/indiv_pair_eval_falsepos.csv')
train_evaluate_lang_fn.to_csv(f'{prefix}/indiv_pair_eval_falseneg.csv')   
weights.to_csv(f'{prefix}/SVM_weights.csv') 


Evaluating on balanced splits

Training on all langs



,Fast Levenshtein Distance Div Maxlen,Dolgo Prime Distance Div Maxlen,Feature Edit Distance Div Maxlen,Hamming Feature Distance Div Maxlen,Weighted Feature Distance Div Maxlen,Partial Hamming Feature Distance Div Maxlen,plain Levenshtein,DNN_logits,MBERT_cos_sim,XLM_cos_sim,loan_unicode_Latin,loan_unicode_Greek,loan_unicode_Cyrillic,loan_unicode_Arabic,loan_unicode_Devanagari,loan_unicode_Bengali,loan_unicode_Gurmukhi,loan_unicode_Tamil,loan_unicode_Telugu,loan_unicode_Malayalam,loan_unicode_Myanmar,loan_unicode_Chinese,original_unicode_Latin,original_unicode_Greek,original_unicode_Cyrillic,original_unicode_Arabic,original_unicode_Devanagari,original_unicode_Bengali,original_unicode_Gurmukhi,original_unicode_Tamil,original_unicode_Telugu,original_unicode_Malayalam,original_unicode_Myanmar,original_unicode_Chinese
0,-0.557592,-0.396341,0.235268,0.657303,-0.109161,-0.487456,-0.799094,2.296604,-0.006159,0.023581,0.000000,0.000000,0.175009,-0.067950,-0.070042,0.000000,0.000000,-0.002170,0.000000,0.000000,0.000000,0.000000,0.203352,0.000000,0.000000,-0.067950,0.000000,0.000000,0.000000,-0.112584,0.000000,0.000000,0.000000,0.000000



Evaluating on all langs
f1-score :  0.9750519750519752
precision :  0.9722222222222222
recall :  0.9778982485404504
accuracy :  0.9749321077919365
              precision    recall  f1-score   support

           0       0.98      0.97      0.97      2389
           1       0.97      0.98      0.98      2398

    accuracy                           0.97      4787
   macro avg       0.97      0.97      0.97      4787
weighted avg       0.97      0.97      0.97      4787

[[2322   67]
 [  53 2345]]
final report [{'all_language_evaluate': True, 'train_evaluate_pair': False, 'language_pair': 'All_languages', 'class': '0', 'precision': 0.98, 'recall': 0.97, 'f1_score': 0.97, 'support': 2389.0}, {'all_language_evaluate': True, 'train_evaluate_pair': False, 'language_pair': 'All_languages', 'class': '1', 'precision': 0.97, 'recall': 0.98, 'f1_score': 0.98, 'support': 2398.0, 'acc': 0.97}]

Evaluating on Hindi-Persian
f1-score :  0.9802371541501976
precision :  0.96875
recall :  0.992
accuracy

f1-score :  0.9754901960784313
precision :  0.9567307692307693
recall :  0.995
accuracy :  0.974937343358396
              precision    recall  f1-score   support

           0       0.99      0.95      0.97       199
           1       0.96      0.99      0.98       200

    accuracy                           0.97       399
   macro avg       0.98      0.97      0.97       399
weighted avg       0.98      0.97      0.97       399

[[190   9]
 [  1 199]]
final report [{'all_language_evaluate': False, 'train_evaluate_pair': False, 'language_pair': 'Romanian-French', 'class': '0', 'precision': 0.99, 'recall': 0.95, 'f1_score': 0.97, 'support': 199.0}, {'all_language_evaluate': False, 'train_evaluate_pair': False, 'language_pair': 'Romanian-French', 'class': '1', 'precision': 0.96, 'recall': 0.99, 'f1_score': 0.98, 'support': 200.0, 'acc': 0.97}]

Training and evaluating on Hindi-Persian


,Fast Levenshtein Distance Div Maxlen,Dolgo Prime Distance Div Maxlen,Feature Edit Distance Div Maxlen,Hamming Feature Distance Div Maxlen,Weighted Feature Distance Div Maxlen,Partial Hamming Feature Distance Div Maxlen,plain Levenshtein,DNN_logits,MBERT_cos_sim,XLM_cos_sim,loan_unicode_Latin,loan_unicode_Greek,loan_unicode_Cyrillic,loan_unicode_Arabic,loan_unicode_Devanagari,loan_unicode_Bengali,loan_unicode_Gurmukhi,loan_unicode_Tamil,loan_unicode_Telugu,loan_unicode_Malayalam,loan_unicode_Myanmar,loan_unicode_Chinese,original_unicode_Latin,original_unicode_Greek,original_unicode_Cyrillic,original_unicode_Arabic,original_unicode_Devanagari,original_unicode_Bengali,original_unicode_Gurmukhi,original_unicode_Tamil,original_unicode_Telugu,original_unicode_Malayalam,original_unicode_Myanmar,original_unicode_Chinese
0,-1.003404,-0.347152,0.243486,-0.746437,-0.004762,0.295978,0.423882,2.587965,-0.055848,0.112921,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


...

f1-score :  0.9711538461538461
precision :  0.9619047619047619
recall :  0.9805825242718447
accuracy :  0.9707317073170731
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       204
           1       0.96      0.98      0.97       206

    accuracy                           0.97       410
   macro avg       0.97      0.97      0.97       410
weighted avg       0.97      0.97      0.97       410

[[196   8]
 [  4 202]]
final report [{'all_language_evaluate': False, 'train_evaluate_pair': True, 'language_pair': 'Polish-French', 'class': '0', 'precision': 0.98, 'recall': 0.96, 'f1_score': 0.97, 'support': 204.0}, {'all_language_evaluate': False, 'train_evaluate_pair': True, 'language_pair': 'Polish-French', 'class': '1', 'precision': 0.96, 'recall': 0.98, 'f1_score': 0.97, 'support': 206.0, 'acc': 0.97}]

Training and evaluating on Indonesian-Dutch


,Fast Levenshtein Distance Div Maxlen,Dolgo Prime Distance Div Maxlen,Feature Edit Distance Div Maxlen,Hamming Feature Distance Div Maxlen,Weighted Feature Distance Div Maxlen,Partial Hamming Feature Distance Div Maxlen,plain Levenshtein,DNN_logits,MBERT_cos_sim,XLM_cos_sim,loan_unicode_Latin,loan_unicode_Greek,loan_unicode_Cyrillic,loan_unicode_Arabic,loan_unicode_Devanagari,loan_unicode_Bengali,loan_unicode_Gurmukhi,loan_unicode_Tamil,loan_unicode_Telugu,loan_unicode_Malayalam,loan_unicode_Myanmar,loan_unicode_Chinese,original_unicode_Latin,original_unicode_Greek,original_unicode_Cyrillic,original_unicode_Arabic,original_unicode_Devanagari,original_unicode_Bengali,original_unicode_Gurmukhi,original_unicode_Tamil,original_unicode_Telugu,original_unicode_Malayalam,original_unicode_Myanmar,original_unicode_Chinese
0,-0.037558,-0.097602,-0.070104,0.983875,-0.828235,0.332257,-1.568361,2.736221,0.075260,0.393812,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


f1-score :  0.9868173258003766
precision :  0.9924242424242424
recall :  0.9812734082397003
accuracy :  0.9868913857677902
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       267
           1       0.99      0.98      0.99       267

    accuracy                           0.99       534
   macro avg       0.99      0.99      0.99       534
weighted avg       0.99      0.99      0.99       534

[[265   2]
 [  5 262]]
final report [{'all_language_evaluate': False, 'train_evaluate_pair': True, 'language_pair': 'Indonesian-Dutch', 'class': '0', 'precision': 0.98, 'recall': 0.99, 'f1_score': 0.99, 'support': 267.0}, {'all_language_evaluate': False, 'train_evaluate_pair': True, 'language_pair': 'Indonesian-Dutch', 'class': '1', 'precision': 0.99, 'recall': 0.98, 'f1_score': 0.99, 'support': 267.0, 'acc': 0.99}]

Training and evaluating on Romanian-French


,Fast Levenshtein Distance Div Maxlen,Dolgo Prime Distance Div Maxlen,Feature Edit Distance Div Maxlen,Hamming Feature Distance Div Maxlen,Weighted Feature Distance Div Maxlen,Partial Hamming Feature Distance Div Maxlen,plain Levenshtein,DNN_logits,MBERT_cos_sim,XLM_cos_sim,loan_unicode_Latin,loan_unicode_Greek,loan_unicode_Cyrillic,loan_unicode_Arabic,loan_unicode_Devanagari,loan_unicode_Bengali,loan_unicode_Gurmukhi,loan_unicode_Tamil,loan_unicode_Telugu,loan_unicode_Malayalam,loan_unicode_Myanmar,loan_unicode_Chinese,original_unicode_Latin,original_unicode_Greek,original_unicode_Cyrillic,original_unicode_Arabic,original_unicode_Devanagari,original_unicode_Bengali,original_unicode_Gurmukhi,original_unicode_Tamil,original_unicode_Telugu,original_unicode_Malayalam,original_unicode_Myanmar,original_unicode_Chinese
0,-0.503193,-0.155482,-0.185494,0.212373,-0.333702,0.298957,-1.215673,1.694677,0.079763,-0.088232,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


f1-score :  0.977886977886978
precision :  0.961352657004831
recall :  0.995
accuracy :  0.9774436090225563
              precision    recall  f1-score   support

           0       0.99      0.96      0.98       199
           1       0.96      0.99      0.98       200

    accuracy                           0.98       399
   macro avg       0.98      0.98      0.98       399
weighted avg       0.98      0.98      0.98       399

[[191   8]
 [  1 199]]
final report [{'all_language_evaluate': False, 'train_evaluate_pair': True, 'language_pair': 'Romanian-French', 'class': '0', 'precision': 0.99, 'recall': 0.96, 'f1_score': 0.98, 'support': 199.0}, {'all_language_evaluate': False, 'train_evaluate_pair': True, 'language_pair': 'Romanian-French', 'class': '1', 'precision': 0.96, 'recall': 0.99, 'f1_score': 0.98, 'support': 200.0, 'acc': 0.98}]

Exists


# Random Forest Classifier 

In [ ]:
def rf_evaluate(full_train_set, full_test_set, pairs_list): 
    all_result = []
    all_lang_fp_list= []
    all_lang_fn_list=[]
    evaluate_lang_fp_list = []
    evaluate_lang_fn_list = []
    train_evaluate_lang_fp_list = []
    train_evaluate_lang_fn_list = []
    pair =None #pair None for all languages evaluation
    display_fields = ['loan_word',
              'original_word',
              'loan_word_epitran',
              'original_word_epitran',
              'loan_english',
              'original_english'] + features
        
    train_set = pd.concat([full_train_set[pair][display_fields + ["label"] + labels] for pair in pairs])
    x_train = train_set[features].values
    x_means = np.mean(x_train, axis=0)
    x_stds = np.std(x_train, axis=0)
    x_stds[x_stds == 0] = 1
    y_train = train_set[labels].values.ravel()
    
    test_set = pd.concat([full_test_set[pair][display_fields + ["label"] + labels] for pair in pairs])
    x_test = test_set[features].values
    y_test = test_set[labels].values.ravel()
    
    # standardize input features
    x_train = (x_train - x_means)/x_stds

    print("Training on all langs\n")
     
    RF = RandomForestClassifier(n_estimators=1000, max_depth=5, random_state=0)
    RF.fit(x_train, y_train)
#     num_leafs = [1, 5, 10, 20, 50, 100]
#     parameters3 = [{'n_estimators' : range(100,800,20),
#              'max_depth': range(1,20,5),
#              'min_samples_leaf':num_leafs
#              }]
    
#     gs3 = GridSearchCV(estimator=RF,
#                   param_grid=parameters3,
#                   cv = 10,
#                   n_jobs = -1)

#     gs3.fit(x_train, y_train)
    
    # display regressor weights
    #df = pd.DataFrame(RF.best_params_, columns=features).style.set_caption('Weights')
    #display(df)
    print()
    
    print("Evaluating on all langs")
    x_test = (x_test - x_means)/x_stds
    y_pred = RF.predict(x_test)

    print("f1-score : ", f1_score(y_test, y_pred ))
    print("precision : ",precision_score(y_test, y_pred))
    print("recall : ",recall_score(y_test, y_pred )) 
    print("accuracy : ",accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    class_report = classification_report(y_test, y_pred)
    a = classification_report_csv(class_report,pair,all_lang=True, train_evaluate_pair =False)
    all_result.append(a)
    print()
    
    # display false positives and false negatives
    unq = np.array([x + 2*y for x, y in zip(y_pred, y_test)])
    tp = np.array(np.where(unq == 3)).tolist()[0]
    fp = np.array(np.where(unq == 1)).tolist()[0]
    tn = np.array(np.where(unq == 0)).tolist()[0]
    fn = np.array(np.where(unq == 2)).tolist()[0]

    display(test_set.reset_index(drop=True).iloc[fp,:][display_fields + ["label"]].style.set_caption('False positives'))
    display(test_set.reset_index(drop=True).iloc[fn,:][display_fields + ["label"]].style.set_caption('False negatives'))
    all_lang_fp = test_set.reset_index(drop=True).iloc[fp,:][display_fields + ["label"]]
    all_lang_fn = test_set.reset_index(drop=True).iloc[fn,:][display_fields + ["label"]]
    all_lang_fp.insert(0, 'language_pair', pair)
    all_lang_fn.insert(0, 'language_pair', pair)
    all_lang_fp_list.append(all_lang_fp)
    all_lang_fn_list.append(all_lang_fn)
    
    for pair in pairs_list:
        print(f'Evaluating on {pair}')
        
        test_set = full_test_set[pair][display_fields + ["label"] + labels]
        x_test = test_set[features].values
        y_test = test_set[labels].values.ravel()
        
        x_test = (x_test - x_means)/x_stds
        y_pred = RF.predict(x_test)
        
        print("f1-score : ", f1_score(y_test, y_pred ))
        print("precision : ",precision_score(y_test, y_pred))
        print("recall : ",recall_score(y_test, y_pred )) 
        print("accuracy : ",accuracy_score(y_test, y_pred))
        print(classification_report(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))
        class_report = classification_report(y_test, y_pred)
        a = classification_report_csv(class_report,pair,all_lang=False, train_evaluate_pair =False)
        all_result.append(a)
        print()
        
        # display false positives and false negatives
        unq = np.array([x + 2*y for x, y in zip(y_pred, y_test)])
        tp = np.array(np.where(unq == 3)).tolist()[0]
        fp = np.array(np.where(unq == 1)).tolist()[0]
        tn = np.array(np.where(unq == 0)).tolist()[0]
        fn = np.array(np.where(unq == 2)).tolist()[0]

        display(test_set.reset_index(drop=True).iloc[fp,:][display_fields + ["label"]].style.set_caption('False positives'))
        display(test_set.reset_index(drop=True).iloc[fn,:][display_fields + ["label"]].style.set_caption('False negatives'))
        
        evaluate_lang_fp = test_set.reset_index(drop=True).iloc[fp,:][display_fields + ["label"]]
        evaluate_lang_fn = test_set.reset_index(drop=True).iloc[fn,:][display_fields + ["label"]]
        evaluate_lang_fp.insert(0, 'language_pair', pair)
        evaluate_lang_fn.insert(0, 'language_pair', pair)
        evaluate_lang_fp_list.append(evaluate_lang_fp)
        evaluate_lang_fn_list.append(evaluate_lang_fn)
        
    for pair in pairs_list:
        print(f'Training and evaluating on {pair}')
        
        train_set = full_train_set[pair][display_fields + ["label"] + labels]
        x_train = train_set[features].values
        x_means = np.mean(x_train, axis=0)
        x_stds = np.std(x_train, axis=0)
        x_stds[x_stds == 0] = 1
        y_train = train_set[labels].values.ravel()
        
        test_set = full_test_set[pair][display_fields + ["label"] + labels]
        x_test = test_set[features].values
        y_test = test_set[labels].values.ravel()
        
        x_train = (x_train - x_means)/x_stds
        
        RF = RandomForestClassifier(n_estimators=1000, max_depth=5, random_state=0)
        RF.fit(x_train, y_train)
#         num_leafs = [1, 5, 10, 20, 50, 100]
#         parameters3 = [{'n_estimators' : range(100,800,20),
#              'max_depth': range(1,20,5),
#              'min_samples_leaf':num_leafs
#              }]
    
#         gs3 = GridSearchCV(estimator=RF,
#                   param_grid=parameters3,
#                   cv = 10,
#                   n_jobs = -1)

#         gs3.fit(x_train, y_train)
        
        # display regressor weights
        #df = pd.DataFrame(RF.best_params_, columns=features).style.set_caption('Weights')
        #display(df)

        x_test = (x_test - x_means)/x_stds
        y_pred = RF.predict(x_test)
        
        print("f1-score : ", f1_score(y_test, y_pred ))
        print("precision : ",precision_score(y_test, y_pred))
        print("recall : ",recall_score(y_test, y_pred )) 
        print("accuracy : ",accuracy_score(y_test, y_pred))
        print(classification_report(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))
        class_report = classification_report(y_test, y_pred)
        a = classification_report_csv(class_report,pair,all_lang=False,train_evaluate_pair =True)
        all_result.append(a)
        print()
        
        # display false positives and false negatives
        unq = np.array([x + 2*y for x, y in zip(y_pred, y_test)])
        tp = np.array(np.where(unq == 3)).tolist()[0]
        fp = np.array(np.where(unq == 1)).tolist()[0]
        tn = np.array(np.where(unq == 0)).tolist()[0]
        fn = np.array(np.where(unq == 2)).tolist()[0]

        display(test_set.reset_index(drop=True).iloc[fp,:][display_fields + ["label"]].style.set_caption('False positives'))
        display(test_set.reset_index(drop=True).iloc[fn,:][display_fields + ["label"]].style.set_caption('False negatives'))
        train_evaluate_lang_fp = test_set.reset_index(drop=True).iloc[fp,:][display_fields + ["label"]]
        train_evaluate_lang_fn = test_set.reset_index(drop=True).iloc[fn,:][display_fields + ["label"]]
        train_evaluate_lang_fp.insert(0, 'language_pair', pair)
        train_evaluate_lang_fn.insert(0, 'language_pair', pair)
        train_evaluate_lang_fp_list.append(train_evaluate_lang_fp)
        train_evaluate_lang_fn_list.append(train_evaluate_lang_fn)
    final_result = pd.concat(all_result, ignore_index=True)
    all_lang_fp_list = pd.concat(all_lang_fp_list, ignore_index=True)
    all_lang_fn_list = pd.concat(all_lang_fn_list, ignore_index=True)
    evaluate_lang_fp_list = pd.concat(evaluate_lang_fp_list, ignore_index=True)
    evaluate_lang_fn_list = pd.concat(evaluate_lang_fn_list, ignore_index=True)
    train_evaluate_lang_fp_list = pd.concat(train_evaluate_lang_fp_list, ignore_index=True)
    train_evaluate_lang_fn_list = pd.concat(train_evaluate_lang_fn_list, ignore_index=True)
        
    return final_result, all_lang_fp_list, all_lang_fn_list, evaluate_lang_fp_list, evaluate_lang_fn_list, train_evaluate_lang_fp_list, train_evaluate_lang_fn_list

In [ ]:
print("Evaluating and saving csvs on alldata splits\n")

output_alldata,all_lang_fp, all_lang_fn, evaluate_lang_fp, evaluate_lang_fn, train_evaluate_lang_fp, train_evaluate_lang_fn = rf_evaluate(train_alldata, test_alldata, pairs)
prefix = f'../Final_results/all_data/RF_results'
if os.path.isdir(prefix):
    print("Exists")
else:
    print("Doesn't exist")
    os.mkdir(prefix)
output_alldata.to_csv(f'{prefix}/final_results.csv')
all_lang_fp.to_csv(f'{prefix}/all_lang_falsepos.csv')
all_lang_fn.to_csv(f'{prefix}/all_lang_falseneg.csv')
evaluate_lang_fp.to_csv(f'{prefix}/all_lang_pair_eval_falsepos.csv')
evaluate_lang_fn.to_csv(f'{prefix}/all_lang_pair_eval_falseneg.csv')
train_evaluate_lang_fp.to_csv(f'{prefix}/indiv_pair_eval_falsepos.csv')
train_evaluate_lang_fn.to_csv(f'{prefix}/indiv_pair_eval_falseneg.csv')



# print("Evaluating on realdist splits\n")
# rf_evaluate(train_realdist, test_realdist, pairs)

# print("Evaluating on balanced splits\n")
# rf_evaluate(train_balanced, test_balanced, pairs)

In [ ]:
print("Evaluating on realdist splits\n")
output_alldata,all_lang_fp, all_lang_fn, evaluate_lang_fp, evaluate_lang_fn, train_evaluate_lang_fp, train_evaluate_lang_fn = rf_evaluate(train_realdist, test_realdist, pairs)
prefix = f'../Final_results/real_dist/RF_results'
if os.path.isdir(prefix):
    print("Exists")
else:
    print("Doesn't exist")
    os.mkdir(prefix)
output_alldata.to_csv(f'{prefix}/final_results.csv')
all_lang_fp.to_csv(f'{prefix}/all_lang_falsepos.csv')
all_lang_fn.to_csv(f'{prefix}/all_lang_falseneg.csv')
evaluate_lang_fp.to_csv(f'{prefix}/all_lang_pair_eval_falsepos.csv')
evaluate_lang_fn.to_csv(f'{prefix}/all_lang_pair_eval_falseneg.csv')
train_evaluate_lang_fp.to_csv(f'{prefix}/indiv_pair_eval_falsepos.csv')
train_evaluate_lang_fn.to_csv(f'{prefix}/indiv_pair_eval_falseneg.csv')

In [ ]:
print("Evaluating on balanced splits\n")
output_alldata,all_lang_fp, all_lang_fn, evaluate_lang_fp, evaluate_lang_fn, train_evaluate_lang_fp, train_evaluate_lang_fn = rf_evaluate(train_balanced, test_balanced, pairs)
prefix = f'../Final_results/balanced/RF_results'
if os.path.isdir(prefix):
    print("Exists")
else:
    print("Doesn't exist")
    os.mkdir(prefix)
output_alldata.to_csv(f'{prefix}/final_results.csv')
all_lang_fp.to_csv(f'{prefix}/all_lang_falsepos.csv')
all_lang_fn.to_csv(f'{prefix}/all_lang_falseneg.csv')
evaluate_lang_fp.to_csv(f'{prefix}/all_lang_pair_eval_falsepos.csv')
evaluate_lang_fn.to_csv(f'{prefix}/all_lang_pair_eval_falseneg.csv')
train_evaluate_lang_fp.to_csv(f'{prefix}/indiv_pair_eval_falsepos.csv')
train_evaluate_lang_fn.to_csv(f'{prefix}/indiv_pair_eval_falseneg.csv')